 <font size="6">Explainability</font>

# Причины использования Explainability

Модели машинного обучения представляют собой черный ящик.

<img src="https://edunet.kea.su/repo/EduNet-content/L10/out/input_blackbox_output.png" alt="alttext" width=400/>

Иногда это  становится препятствием для внедрения моделей.

Есть как минимум три причины, по которым нас может интересовать объяснение предсказаний модели.

## Обнаружение некорректных зависимостей 

Модель может использовать совсем не те признаки, которые соответствуют реальным объектам.

Например, ориентироваться на фон или водяной знак, а не на реальные свойства объекта.

Пример из статьи ["Why Should I Trust You?"](https://arxiv.org/abs/1602.04938) 
Авторы обучили классификатор волков и эскимосских собак (хаски). Исследователи на изображениях, отобранных так, чтобы на всех фотографиях волков на фоне был снег, а на фотографиях хаски — нет.

<center><img src="https://edunet.kea.su/repo/EduNet-web_dependencies/L10/2_bad_models_prediction.png" alt="alttext" width=400/></center>

<center><p><em>Source: <a href="https://arxiv.org/pdf/1602.04938.pdf">Explaining the Predictions of Any Classifier</a></p> </em></center>

## Доверие к предсказаниям 

<img src="https://edunet.kea.su/repo/EduNet-web_dependencies/L10/nuclear.jpg" alt="alttext" width=650/>

Нельзя остановить ядерную электростанцию или назначить пациенту опасное лечение на основании предсказания «черного ящика». 
Даже маловероятная ошибка в таких случаях будет иметь тяжелые последствия. Поэтому человек, принимающий решение, должен понимать на основе каких признаков или симптомов сделано предсказание.

## Публикации в научных журналах

Вероятность публикации статьи значительно повышается, если автор смог объяснить происхождение результатов своего исследования.

<img src="https://edunet.kea.su/repo/EduNet-content/L10/out/scientific_publication.png" alt="alttext" width=650/>

## Explainability & Interpretability

В англоязычной литературе можно встретить два термина, связанных с темой доверия: Explainability и Interpretability

**Explainability** — методики, позволяющие объяснить механизм функционирования модели. 

Например, для линейной регрессии это анализ коэффициентов при параметрах.


**Interpretability** — анализ того, как изменение входов модели влияло на ее выходы.

Например, закрашивая часть пикселей изображения, можно выяснить, какие из них повлияют на предсказания (пример с хаски и волками).


В данном блокноте будем преимущественно рассматривать методы второго типа.

[Machine Learning Explainability vs Interpretability: Two concepts that could help restore trust in AI](https://www.kdnuggets.com/2018/12/machine-learning-explainability-interpretability-ai.html)



<img src="https://edunet.kea.su/repo/EduNet-content/L10/out/data_pipeline.png" alt="alttext" width=850/>


# Оценка важности признаков в простых случаях

## Оценка важности признака для линейных моделей



В случае с **линейными моделями** нам было сравнительно легко определить, какие признаки модель считает важными. Если модель присваивает какому-то признаку **большой вес**, то этот признак сильнее влияет на **предсказание**.


###Пример для табличных данных (Boston Dataset)

Для примера скачаем **датасет жилья Бостона** (boston_dataset), в котором проанализируем зависимость цены на жилье от параметров жилья и района, в котором оно находится.

In [ ]:
import pandas as pd

# load dataset
boston_dataset = pd.read_csv(
    "https://edunet.kea.su/repo/EduNet-web_dependencies/L10/boston_dataset.csv",
    index_col=0,
)
x_data = boston_dataset.iloc[:, :-1]
y_data = boston_dataset["target"].values

boston_dataset.head()

Обучим модель:

In [ ]:
from sklearn.linear_model import LinearRegression

model = LinearRegression()
model.fit(x_data, y_data)

Выведем **коэффициенты** признаков:

In [ ]:
df = pd.DataFrame({"name": x_data.columns, "coef": model.coef_})
df

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns

df["colors"] = ["red" if x < 0 else "lawngreen" for x in df["coef"]]

plt.figure(figsize=(8, 8))
sns.barplot(data=df, y="name", x="coef", palette=df.colors, orient="h")
plt.show()

In [ ]:
from sklearn.metrics import mean_squared_error

mean_squared_error(y_data, model.predict(x_data))

Посмотрим на статистику признаков:

In [ ]:
boston_dataset.describe()

Видно, что самый “важный” признак `NOX` имеет стандартное отклонение 0.115878, а, например, `LSTAT` имеет стандартное отклонение на порядок больше. Это значит, что веса перед признаком будут не только показывать важность, но и масштабировать признаки. Чтобы избежать этого, нужно сделать нормализацию. 


Посмотрим на **коэффициенты** после **нормализации** данных.

In [ ]:
# load dataset
boston_dataset = pd.read_csv(
    "https://edunet.kea.su/repo/EduNet-web_dependencies/L10/boston_dataset.csv",
    index_col=0,
)
x_data = boston_dataset.iloc[:, :-1]
y_data = boston_dataset["target"].values

In [ ]:
from sklearn.preprocessing import StandardScaler

ss = StandardScaler()
x_data = ss.fit_transform(x_data)

In [ ]:
model = LinearRegression()
model.fit(x_data, y_data)

In [ ]:
df = pd.DataFrame({"name": boston_dataset.columns[:-1], "coef": model.coef_})
df

In [ ]:
df["colors"] = ["red" if x < 0 else "lawngreen" for x in df["coef"]]

plt.figure(figsize=(8, 8))
sns.barplot(data=df, y="name", x="coef", palette=df.colors, orient="h")
plt.show()

In [ ]:
mean_squared_error(y_data, model.predict(x_data))

Как мы видим, **качество модели** не изменилось, зато новая оценка позволяет сравнивать признаки по **важности**, а не по диапазону значений. 

## Оценка важности признака для дерева

В случае же с **деревьями** всё далеко не так очевидно: дерево не знает такой концепции, как **"вес признака"**.

Универсального критерия значимости для деревьев у нас нет, и в зависимости от задачи и от того, как эти признаки устроены, ответы могут быть разными.

С одним из способов оценки качества признаков для дерева решений мы познакомились, когда строили дерево решений на третьей лекции. Мы использовали метрику $Gini$:

$$Gini = 1 - \sum_ip_i^2$$

где $p_i$ — вероятность того, что объект, попавший в данный  лист, относится к $i$-ому классу. Чем меньше $Gini$ в листьях, тем качественнее узел, от которого “растут” листья, разделяет классы. 


Для того, чтобы охарактеризовать “хорошесть” узла, мы строили метрику $impurity$, в которой суммировали $Gini$ листьев данного узла с весами, равными доле объектов, попавших в данный лист. После чего смотрели, на сколько $impurity$ уменьшилось ($decrease$) на данном узле (стало ближе к “идеальному” нулю). 


<img src="https://edunet.kea.su/repo/EduNet-content/L10/out/gini_impurity.png" >

Именно $impurity$ $decrease$ используется для расчета атрибута `feature_importances_` в [sklearn](https://scikit-learn.org/stable/auto_examples/ensemble/plot_forest_importances.html). Такая метрика встроена в любое дерево решений. Для случайного леса (и других ансамблей) просто выдается среднее по деревьям.


У данного метода есть **недостаток**: он завышает качество признаков с большим количеством возможных значений. 

Даже если признак не информативен, но у него много значений, на нем можно выбрать большое количество порогов, по которым можно разбить данные (что приведет к переобучению, но $Gini$ об этом не знает). 

Бинарные и маломерные категориальные признаки в любом случае будут получать заниженное качество по сравнению с вещественными, даже если те дают худшее разбиение.

<img src="https://edunet.kea.su/repo/EduNet-content/L10/out/gini_impurity_disadvantage.png" >

### Пример для табличных данных (Boston Dataset)

Посмотрим на важность признаков, основанную на impurity decrease.

In [ ]:
import numpy as np
from sklearn.ensemble import RandomForestRegressor

rng = np.random.RandomState(42)
model = RandomForestRegressor(random_state=rng)
model.fit(x_data, y_data)

In [ ]:
df = pd.DataFrame(
    {"name": boston_dataset.columns[:-1], "imp": model.feature_importances_}
)
df

In [ ]:
plt.figure(figsize=(8, 8))
sns.barplot(data=df, y="name", x="imp", color="blue", orient="h")
plt.show()

## Randomization/Permutation



Для определения важности признака можно перемешать его значения. Это не изменит форму распределения признака, но сделает его бессмысленным. По падению качества модели мы можем оценить, каков был вклад признака до изменения: если качество упало сильно, то он был значимым.

В sklearn это реализовано как отдельный класс [`permutation_importance`](https://scikit-learn.org/stable/modules/permutation_importance.html).

Изменение качества модели рассчитывается на OOB (**Out-of-bag samples**) объектах: на объектах, которые не вошли в обучающую выборку, а остались для валидации (не путать с тестовой выборкой, которую мы не трогаем). 

У данного метода есть три особенности:
1. Значение **важности признака** будет зависеть от **результата перемешивания**, которое носит случайный характер. Поэтому функция `permutation_importance` имеет параметр `n_repeats`, который определяет, сколько раз признак будет перемешиваться случайным образом. 
2. Значение **важности признака** будет зависеть от **модели** и того, насколько “удачно” она обучена. “Плохая” и “хорошая” модель может считать важным разный набор признаков. Поэтому полезно сравнить важность признаков модели, обучив модель на различных выборках (например, используя cross-validation).
3. Данный метод неадекватно оценивает **важность коррелированных признаков**.

Когда два признака сильно **коррелированы** и мы перемешали один из них, **линейная модель** сохранит информацию о перемешанном признаке через коррелированный с ним, что может привести к **занижению важности**. 

[[Doc sklearn] Misleading values on strongly correlated features](https://scikit-learn.org/stable/modules/permutation_importance.html#misleading-values-on-strongly-correlated-features)

С другой стороны, может произойти обратный эффект. 
**Дерево решений** пытается разбить пространство плоскостями, и в областях, где объектов нет, оно, можно сказать, занимается угадыванием. Если $x_1$ и $x_2$ на картинке ниже линейно зависимы, то не может возникнуть ситуация, при которой $x_1 = 0$, а $x_2 = 1$. А как раз при перемешивании такая ситуация возникнет, и точки начнут попадать в "проблемные" области, в которых дерево решений плохо предсказывает. В результате получаем **завышенную важность**.

<img src="https://edunet.kea.su/repo/EduNet-web_dependencies/L10/quality_out_of_bag_samples.png" alt="alttext" width=800/>

<center><p><em>Source: <a href="https://arxiv.org/pdf/1905.03151.pdf">Unrestricted Permutation forces Extrapolation: Variable Importance Requires at least One More Model oThere Is No Free Variable Importance
</a></p> </em></center>

Для борьбы с неадекватной оценкой **коррелированных признаков** можно собирать коррелированные признаки в кластеры и оставить  только один признак для каждого [кластера](https://scikit-learn.org/stable/auto_examples/inspection/plot_permutation_importance_multicollinear.html#sphx-glr-auto-examples-inspection-plot-permutation-importance-multicollinear-py).

Одна из вариаций `permutation importance` подразумевает **обучение модели заново** после каждого **перемешивания признаков**. Это дает чуть более адекватную оценку важности, но очень долго. 

### Пример для табличных данных (Boston Dataset)

Сделаем `permutation importance`:

1. Обучим модель
2. Перемешаем значения в одном столбце (одного признака), сделаем предсказание на валидационном наборе данных и посчитаем ошибку. Ухудшение ошибки покажет важность признака, который перемешали. 
3. Возвращаем значения признака и повторяем шаг 2 со всеми остальными признаками

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.inspection import permutation_importance

x_train, x_val, y_train, y_val = train_test_split(x_data, y_data, random_state=rng)
model = RandomForestRegressor(random_state=rng)
model.fit(x_train, y_train)

r = permutation_importance(model, x_val, y_val, n_repeats=100, random_state=rng)

df = pd.DataFrame({"name": boston_dataset.columns[:-1], "imp": r.importances_mean})
df

In [ ]:
plt.figure(figsize=(8, 8))
sns.barplot(data=df, y="name", x="imp", color="blue", orient="h")
plt.show()

Ниже выполняются те же операции, но уже с линейной регрессией.

In [ ]:
x_train, x_val, y_train, y_val = train_test_split(x_data, y_data, random_state=rng)
model = LinearRegression()
model.fit(x_train, y_train)

r = permutation_importance(model, x_val, y_val, n_repeats=100, random_state=rng)

df = pd.DataFrame({"name": boston_dataset.columns[:-1], "imp": r.importances_mean})
df

In [ ]:
plt.figure(figsize=(8, 8))
sns.barplot(data=df, y="name", x="imp", color="blue", orient="h")
plt.show()

Как видно, у некоторых признаков при переходе от древесных моделей к линейным значительно повысилась важность, что говорит об их скоррелированности.

## Dropped variable importance

Редко используемый метод, заключающийся в том, что мы просто выбрасываем переменную и смотрим, как упало качество по сравнению с "полной" моделью.

Это не очень хорошо как минимум потому, что многие гиперпараметры зависят от числа признаков $-$ модель будет работать хуже еще и поэтому. Опять же, между признаками могут быть сложные взаимодействия.

# Методы, изучающие отклик модели на изменение входных данных

Мы рассмотрим две библиотеки:
- SHAP
- LIME

**SHAP (SHapley Additive exPlanations)** — подход к объяснению моделей машинного обучения, основанный на теории игр. Интерпретация результатов модели SHAP основана на оценке локальной точности признаков (local importance) для каждого объекта, которая определяется [значением Шепли](https://en.wikipedia.org/wiki/Shapley_value). Значения Шепли можно интерпретировать как важность признака для качества предсказания модели, включающей данный признак, по сравнению с моделью, не включающей его, для каждой комбинации признаков.

Достоинство SHAP — единый подход ко всем признакам и всем объектам внутри датасета, что обеспечивает глобальную интерпретацию результата.

Ограничением SHAP является то, что для обучения моделей на основе всех комбинаций признаков требуется много вычислительных ресурсов.

**LIME (Local Interpretable Model-agnostic Explanations)** — более экономичная альтернатива для получения локальной интерпретации. Алгоритм LIME применяется к предсказанию обученной модели на одном объекте.

Каждый признак объекта многократно слабо изменяют и получают предсказание модели для таких "искаженных" объектов. В результате формируют карту предсказаний в малой области вокруг объекта. Эту карту аппроксимируют одной из простейших интерпретируемых моделей.

Достоинство LIME — высокая скорость работы. В ходе анализа нет необходимости многократно обучать сложные модели, можно быстро получить локальную интерпретацию результата на интересующем исследователя объекте. Иногда этого вполне достаточно.

## LIME

Значения Шепли, используемые SHAP, которые были показаны в предыдущем примере, хорошо подходят для объяснения вклада в предсказание небольшого количества признаков. Однако SHAP работает не совсем корректно в моделях, использующих множество признаков, поскольку он обычно приписывает ненулевые вклады для всех доступных переменных. В таких сферах, как, например, биоинформатика или распознавание изображений, часто модели используют тысячи и миллионы признаков. В таких случаях была бы полезна процедура прореживания входных признаков до некоего небольшого числа. Наиболее популярный подход, позволяющий это делать — [__LIME__](https://arxiv.org/abs/1602.04938) (Local Interpretable Model-agnostic Explanations).

Ключевая идея LIME — локальная аппроксимация сложно-интерпретируемой (black-box) модели при помощи легко-интерпретируемой (glass-box) модели (например, линейной моделью с несколькими ненулевыми коэффициентами), получающаяся при внесении изменений в данные. 

Это оказывается преимуществом с точки зрения интерпретируемости, потому что мы можем вносить изменения в признаки на входе, которые имеют смысл для людей (например, слова или части изображения), даже если модель использует гораздо более сложные компоненты в качестве функций (например, word embeddings).

### Принцип работы

Принцип работы LIME можно проиллюстрировать следующим изображением:

<center><img src ="https://edunet.kea.su/repo/EduNet-content/L10/out/lime_example.png" width="600"></center>

Допустим, мы хотим понять признаки, влияющие на предсказание некоторой black-box модели для некоторого образца (фиолетовый крест). По осям отложены значения двух непрерывных признаков. Граница между цветными зонами — разделяющая поверхность бинарного классификатора, использующего комбинацию этих 2-х признаков (фиолетовый цвет — кресты, голубой — круги). Для понимания того, как ведет себя сложная black-box модель в окрестности точки интереса, мы генерируем искусственный набор данных, представленный объектами меньшего размера вокруг фиолетового креста (размер зависит от близости к объекту интереса), на котором обучаем glass-box модель, которая будет локально аппроксимировать (пунктирная линия) предсказание black-box модели. В качестве glass-box моделей могут быть использованы, например, LASSO регрессия или решающие деревья.

### Описание алгоритма

Итак, мы хотим найти glass-box модель $g(x)$, которая будет локально аппроксимировать black-box модель $f(x)$ в окрестности объекта интереса $x^*$. $G$ — семейство интерпретируемых моделей (линейные модели, деревья, ..). Искомая аппроксимация будет выглядеть следующим образом:

$$\hat{g}=\underset{g\in G}{\mathrm{argmin}}L(f,g,\pi_x)+\Omega (g),$$

где $\pi_x$ — расстояние, определяющееся по некоторой метрике от сгенерированных объектов до объекта интереса, функция $L()$ измеряет несоответствие между предсказаниями моделей $f()$ и $g()$, а $\Omega (g)$ — штраф за сложность модели  $g()$. На практике зачастую, чтобы не заниматься оптимизацией этого штрафа, просто вводят некоторое ограничение на сложность моделей $g()$. Для деревьев это может быть глубина дерева, для линейных моделей — число ненулевых коэффициентов.

В начале мы упоминали, что LIME оперирует неким разреженным набором признаков, соответственно, стоит обратить внимание на то, что модели $f()$ и $g()$ оперируют разными пространствами признаков, $f()$ — пространством размерности $p$ ($R^p$), соответствующей количеству признаков в исходных данных, $g()$ — пространством размерности $q$ ($R^q$), при этом $q<<p$. Пространство $R^q$ называется _интерпретируемым представлением_ признаков. Пусть некая функция $h(x)$ переводит пространство признаков $R^p$ в $R^q$.

Саму функцию, оценивающую расхождение между предсказаниями моделей $f()$ и $g()$, можно представить следующим образом:

$$L(f,g,\pi_x)=\sum_{z,z'}\pi_x(z)(f(z)-g(z'))^2,$$

где $z$ и $z'$ — наборы искусственно сгенерированных объектов в окрестности $x^*$ в пространствах $R^p$ и $R^q$ соответственно.

Теперь мы можем собрать целиком алгоритм получения объяснения вклада переменных. Представим его в виде псевдокода:

1. Дано: $x^*$ — объект для интерпретации вкладов признаков в модель, $N$ — размер искусственного датасета в окрестности целевого объекта, $similarity$ — метрика расстояния.
2. $x' = h(x^*)$ переведем целевой объект в пространство меньшей размерности

```
z' = []
for i in rnage(N):
    z'[i] = sample_around(x')
    y'[i] = f(z[i])
    pi_x'[i] = similarity(x', z'[i])

return K-Lasso(y', x', pi_x')
```

Выдачей алгоритма служит линейная модель, отобравшая К признаков на основе $y'$, $x'$, $\pi_x'$.

### Как получить набор объектов вблизи искомого?

В зависимости от типа данных, изменения вносятся по-разному:
*  В текстах можно просто удалить слово;
*  Для изображений можно делить картинку на области (суперпиксели) и поочередно закрашивать их одним и тем же цветом (средним).

Проще всего разобраться с тем, как себя вести с табличными данными:

Для бинарных переменных в низкоразмерном пространстве достаточно просто менять значение переменной на противоположное(0 на 1 и 1 на 0). Для вещественных переменных были предложены разные варианты. Например, к ним можно прибавлять Гауссов шум или дискретизировать (например, по квантилям), и затем изменять получившиеся признаки.

### Ограничения

Описанный подход позволяет интерпретировать поведение модели только в некоторой области вблизи интересующего нас экземпляра.
На практике этого может быть достаточно. И, в отличие от SHAP, этот подход работает быстрее, так как не требует перебора всех комбинаций признаков.

### Пример NLP (объяснения классификации статей по религиозному принципу)

В данном примере мы будем использовать две модели из классического NLP, с которыми раньше не работали. Скажем несколько слов о том, как они работают.

#### TF-IDF

**TF-IDF** — способ численного представления документа, оценивает  **важность слова в контексте документа**. Состоит из двух множителей $TF$ и $IDF$.

Первая идея, которая стоит за **TF-IDF** — это **если слово часто встречается в документе, оно важное**. За это отвечает $TF$.

$TF$ (term frequency) — частота вхождения слова в документ, для которого рассчитывается значение:

$$TF(t, d) = \frac{n_t}{\sum_{k}n_k}$$

$n_t$ — количество повторов слова $t$ в документе $d$,

$\sum_{k}n_k$ — общее количество слов  $t$ в документе $d$ с повторами.

Вторая идея **TF-IDF** — **если слово встречается во многих документах, его ценность снижается**. 

Пример: местоимения встречаются в большинстве текстов, но не несут смысловой нагрузки. 

За это отвечает $IDF$.

$IDF$ (inverse document frequency) — логарифм обратной частоты вхождения слова в документы. 
$$IDF(t, D) = \log{\frac{|D|}{|\{d_i \in D| t_i \in d_i \}|}}$$

где $|D|$ — число документов в коллекции,
$|\{d_i \in D| t_i \in d_i \}|$ — число документов в коллекции со словом $t$.


Итоговая формула:
$$TF-IDF(t, d, D) = TF(t,d)⋅IDF(t, D)$$


Пример работы `TfidfVectorizer` из [sklearn](https://scikit-learn.org/stable/modules/generated/sklearn.feature_extraction.text.TfidfVectorizer.html).
Каждому документу сопоставляется вектор, равный длине словаря. Ненулевые значения вектора хранятся в виде разреженных матриц [‘scipy.sparse.csr_matrix’](https://docs.scipy.org/doc/scipy/reference/generated/scipy.sparse.csr_matrix.html).

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer

corpus = [
    "This is the first document.",
    "This document is the second document.",
    "And this is the third one.",
    "Is this the first document?",
]

vectorizer = TfidfVectorizer()
x = vectorizer.fit_transform(corpus)

print("Tf-idf dictionary:", vectorizer.get_feature_names_out())
print("Tf-idf dictionary len:", len(vectorizer.get_feature_names_out()))
print("Tf-idf shape:", x.shape)
print("Tf-idf type:", type(x))
print("Tf-idf values:", x)

TF-IDF (TF — term frequency, IDF — inverse document frequency):


[[wiki] TF-IDF](https://ru.wikipedia.org/wiki/TF-IDF)

[[code] sklearn.feature_extraction.text.TfidfVectorizer](https://scikit-learn.org/stable/modules/generated/sklearn.feature_extraction.text.TfidfVectorizer.html)

[[medium] TF-IDF Vectorizer scikit-learn](https://medium.com/@cmukesh8688/tf-idf-vectorizer-scikit-learn-dbc0244a911a)

#### Задача

<img src="https://edunet.kea.su/repo/EduNet-web_dependencies/L10/christian_or_atheist.png" alt="alttext" width=900/>

<center><p><em>Source: <a href="https://arxiv.org/pdf/1602.04938.pdf">Explaining the Predictions of Any Classifier</a></p> </em></center>

Используем датасет [fetch_20newsgroups](https://scikit-learn.org/stable/modules/generated/sklearn.datasets.fetch_20newsgroups.html): 

Данные [«The 20 Newsgroups»](http://qwone.com/~jason/20Newsgroups/) — это коллекция примерно из 20000 новостных документов, разделенная (приблизительно) равномерно между 20 различными категориями. Изначально она собиралась Кеном Ленгом (Ken Lang), возможно, для его работы «Newsweeder: Learning to filter netnews» («Новостной обозреватель: учимся фильтровать новости из сети»).

Коллекция «The 20 newsgroups» стала популярным набором данных для экспериментов с техниками машинного обучения для текстовых приложений, таких как классификация текста или его кластеризация.

[[code] Fetching data, training a classifier](https://marcotcr.github.io/lime/tutorials/Lime%20-%20multiclass.html)

В данном примере мы будем использовать [Multinomial Naive Bayes](https://scikit-learn.org/stable/modules/generated/sklearn.naive_bayes.MultinomialNB.html?highlight=multinomial%20naive%20bayes#sklearn.naive_bayes.MultinomialNB) для классификации.

In [ ]:
import sklearn
from sklearn.datasets import fetch_20newsgroups

newsgroups_train = fetch_20newsgroups(subset="train")
newsgroups_test = fetch_20newsgroups(subset="test")
# making class names shorter
class_names = [
    x.split(".")[-1] if "misc" not in x else ".".join(x.split(".")[-2:])
    for x in newsgroups_train.target_names
]
class_names[3] = "pc.hardware"
class_names[4] = "mac.hardware"

print(class_names)

LIME используется для объяснения множества классификаторов (таких как RandomForest или SVM и нейронные сети) при анализе моделей NLP и CV.

Знаменитый набор данных из 20 групп новостей является эталоном, он использовался для сравнения различных моделей в нескольких статьях.

Мы берем два класса, которые трудно различить, потому что в них много схожих слов: христианство и атеизм.

Обучая модель, мы получаем точность тестового набора 83,5%, что является удивительно высоким показателем. Если бы точность была нашим единственным мерилом доверия, мы бы точно доверились этому классификатору.

Однако давайте посмотрим на объяснение на рисунке для произвольного экземпляра в тестовом наборе:

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer
import sklearn.metrics
from sklearn.naive_bayes import MultinomialNB

# Again, let's use the tfidf vectorizer, commonly used for text.
vectorizer = sklearn.feature_extraction.text.TfidfVectorizer(lowercase=False)
train_vectors = vectorizer.fit_transform(newsgroups_train.data)
test_vectors = vectorizer.transform(newsgroups_test.data)

# Train the model
model_nb = MultinomialNB(alpha=0.01)
model_nb.fit(train_vectors, newsgroups_train.target)

# Calculate F1_score
pred = model_nb.predict(test_vectors)
sklearn.metrics.f1_score(newsgroups_test.target, pred, average="weighted")

In [ ]:
print(train_vectors.shape)
print(type(train_vectors[0]), train_vectors[0].shape)
print(vectorizer.get_feature_names_out())

Как видно из кода, текст подается на вход модели не в сыром виде, а после предобработки объектом vectorizer. 

[LimeTextExplainer](https://lime-ml.readthedocs.io/en/latest/lime.html#lime.lime_text.LimeTextExplainer)  ждет на вход данные и класс модели:


```
explain_instance(
    text_instance, 
    classifier_fn, 
    labels=(1, ), 
    top_labels=None, 
    num_features=10, 
    num_samples=5000, 
    distance_metric='cosine', 
    model_regressor=None)
```

**classifier_fn** *— classifier prediction probability function, which takes a list of d strings and outputs a (d, k) numpy array with prediction probabilities, where k is the number of classes. For ScikitClassifiers , this is classifier.predict_proba.*



 Что поэтому в примере используется обертка над классом преобразовывающем данные и моделью:

 [sklearn.pipeline.make_pipeline](https://scikit-learn.org/stable/modules/generated/sklearn.pipeline.make_pipeline.html)

In [ ]:
from sklearn.pipeline import make_pipeline

# https://scikit-learn.org/stable/modules/generated/sklearn.pipeline.make_pipeline.html
model_with_preprocessing = make_pipeline(vectorizer, model_nb)

Мы видим, что этот классификатор имеет очень высокий F1_score. Руководство sklearn для 20 newsgroups указывает, что Multinomial Naive Bayes переучивается на этом наборе данных, изучая нерелевантные взаимосвязи, такие как заголовки.

Теперь мы используем LIME для объяснения индивидуальных прогнозов...


В случае мультикласса мы должны определить, для каких меток хотим получить объяснения с помощью параметра «labels». Сгенерируем пояснения для меток 0 и 17:


In [ ]:
!pip -q install lime

In [ ]:
import lime
from lime.lime_text import LimeTextExplainer
import numpy as np

rng = np.random.RandomState(42)
explainer = LimeTextExplainer(class_names=class_names, random_state=rng)
idx = 1340
exp = explainer.explain_instance(
    newsgroups_test.data[idx],
    model_with_preprocessing.predict_proba,
    num_features=6,
    labels=[0, 17],
)
print("Document id: %d" % idx)
print(
    "Predicted class =",
    class_names[model_nb.predict(test_vectors[idx]).reshape(1, -1)[0, 0]],
)
print("True class: %s" % class_names[newsgroups_test.target[idx]])

Возвращается, как и в случае с SHAP, специальный объект класса [Explanation](https://lime-ml.readthedocs.io/en/latest/lime.html?highlight=Explanation#lime.explanation.Explanation)


Попросим LIME сгенерировать метки для K=2 классов. Чтобы увидеть, какие ярлыки имеют объяснения, используйте функцию `available_labels`

In [ ]:
idx = 1340
exp = explainer.explain_instance(
    newsgroups_test.data[idx],
    model_with_preprocessing.predict_proba,
    num_features=6,
    top_labels=2,
)
print(exp.available_labels())

In [ ]:
print(exp.as_list(label=0))
print(exp.as_list(label=15))

Обратите внимание, что положительный и отрицательный знаки относятся к конкретной метке, так что слова, отрицательные по отношению к классу 0, могут быть положительными по отношению к классу 15, и наоборот.

In [ ]:
exp.show_in_notebook(text=newsgroups_test.data[idx], labels=(0,))

Теперь давайте посмотрим на визуализацию объяснений.
Обратите внимание, что для каждого класса слова в правой части строки являются «положительными», а слова в левой части — «отрицательными» для объясняемого класса.

Также видно, что в классификаторе используются как разумные слова (такие как «геноцид», «Лютер», «семитский» и т. д.), так и неразумные («рис», «сова»).

Давайте увеличим масштаб и просто посмотрим на объяснения класса «атеизм».

In [ ]:
exp.show_in_notebook(text=newsgroups_test.data[idx], labels=(15,))

Глядя на этот пример, можно увидеть, что в заголовке или кавычках может быть и полезный сигнал, который будет помогать обобщению (например,  в строке «Тема»).

А есть и слова, которые нельзя обобщать (например, адреса электронной почты и названия учреждений).

###Пример изображения (ResNet18)

[Local Interpretable Model-Agnostic Explanations (LIME): An Introduction](https://www.oreilly.com/content/introduction-to-local-interpretable-model-agnostic-explanations-lime/)

[[git] Using Lime with Pytorch](https://github.com/marcotcr/lime/blob/master/doc/notebooks/Tutorial%20-%20images%20-%20Pytorch.ipynb)

#### Идея


Давайте разберемся, как работает такое эффектное представление областей, по которым нейронная сеть осуществляет предсказание класса.

На рисунке ниже показан пример того, как LIME работает для классификации изображений.

Представьте, что мы хотим объяснить классификатор, который предсказывает, насколько вероятно, что изображение содержит древесную лягушку.

Мы берем изображение слева и делим его на интерпретируемые компоненты (смежные [суперпиксели](https://darshita1405.medium.com/superpixels-and-slic-6b2d8a6e4f08)).

<img src="https://edunet.kea.su/repo/EduNet-web_dependencies/L10/lime_interpretable_components.jpg" alt="alttext" width=550/>

<center><p><em>Source: <a href="https://medium.com/intel-student-ambassadors/local-interpretable-model-agnostic-explanations-lime-the-eli5-way-b4fd61363a5e">Local Interpretable Model-Agnostic Explanations (LIME) — the ELI5 way</a></p> </em></center>

Далее мы отключаем некоторые из суперпикселей (закрашиваем серым).

Для каждой такой картинки мы получаем вероятность того, что на изображении есть древесная лягушка, и формируем датасет из частично закрашенных картинок и предсказаний.

Затем мы обучаем линейную модель на этом наборе данных. Веса, соответствующие суперпикселю, будут объяснять его вклад в предсказание.

<img src="https://edunet.kea.su/repo/EduNet-web_dependencies/L10/lime_explanation_pipeline.jpg" alt="alttext" width=750/>

<center><p><em>Source: <a href="https://medium.com/intel-student-ambassadors/local-interpretable-model-agnostic-explanations-lime-the-eli5-way-b4fd61363a5e">Local Interpretable Model-Agnostic Explanations (LIME) — the ELI5 way</a></p> </em></center>

Проанализируем предсказание сверточной сети Google Inception. Посмотрим, почему ее классификатор предсказывает «древесную лягушку» как наиболее вероятный класс, за которым следуют «бильярдный стол» и «воздушный шар» с более низкими вероятностями.

Мы видим, что классификатор в первую очередь фокусируется на морде лягушки как на объяснении предсказанного класса.

Это также проливает свет на то, почему «бильярдный стол» имеет ненулевую вероятность: руки и глаза лягушки напоминают бильярдные шары, особенно на зеленом фоне. Точно так же сердце похоже на красный воздушный шар.


<img src="https://edunet.kea.su/repo/EduNet-web_dependencies/L10/lime_explaination_google_inception.jpg" alt="alttext" width=750/>

<center><p><em>Source: <a href="https://www.oreilly.com/content/introduction-to-local-interpretable-model-agnostic-explanations-lime/?imm_mid=0e6f40&cmp=em-data-na-na-newsltr_20160824">Local Interpretable Model-Agnostic Explanations (LIME): An Introduction</a></p> </em></center>

[[medium] LIME - Local Interpretable Model-Agnostic Explanation](https://medium.com/intel-student-ambassadors/local-interpretable-model-agnostic-explanations-lime-the-eli5-way-b4fd61363a5e)

#### Анализ ResNet18

In [ ]:
!wget -q 'https://edunet.kea.su/repo/EduNet-web_dependencies/L10/cat_and_dog1.jpg' -O cat_and_dog1.jpg
!wget -q 'https://edunet.kea.su/repo/EduNet-web_dependencies/L10/cat_and_dog2.png' -O cat_and_dog2.png
!wget -q 'https://edunet.kea.su/repo/EduNet-web_dependencies/L10/imagenet_class_index.json' -O imagenet_class_index.json

In [ ]:
import os
from PIL import Image


def get_image(path):
    with open(os.path.abspath(path), "rb") as f:
        with Image.open(f) as img:
            return img.convert("RGB")


img = get_image("cat_and_dog1.jpg")
plt.imshow(img)
plt.show()

Теперь нам нужно преобразовать это изображение в тензор PyTorch и нормализовать его для использования в нашей предварительно обученной модели.

In [ ]:
from torchvision import transforms


# resize & normalize
def get_input_transform():
    normalize = transforms.Normalize(
        mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]
    )
    transform = transforms.Compose(
        [
            transforms.Resize((256, 256)),
            transforms.CenterCrop((224, 224)),
            transforms.ToTensor(),
            normalize,
        ]
    )
    return transform


def get_input_tensors(img):
    transform = get_input_transform()
    # unsqeeze converts single image to batch of 1
    return transform(img).unsqueeze(0)


Загрузим предобученную модель ResNet18, доступную в PyTorch, и классы изображений из ImageNet.

In [ ]:
from torchvision import models
import json

model = models.resnet18(weights="ResNet18_Weights.DEFAULT")

idx2label, cls2label, cls2idx = [], {}, {}
with open(os.path.abspath("/content/imagenet_class_index.json"), "r") as read_file:
    class_idx = json.load(read_file)
    idx2label = [class_idx[str(k)][1] for k in range(len(class_idx))]
    cls2label = {
        class_idx[str(k)][0]: class_idx[str(k)][1] for k in range(len(class_idx))
    }
    cls2idx = {class_idx[str(k)][0]: k for k in range(len(class_idx))}

Получим предсказание. А после этого полученные нами прогнозы (логиты) пропустим через softmax, чтобы получить вероятности и метки классов для 5 лучших прогнозов.

In [ ]:
import torch.nn.functional as F

img_t = get_input_tensors(img)
model.eval()
logits = model(img_t)

probs = F.softmax(logits, dim=1)
probs5 = probs.topk(5)
plt.imshow(img)
plt.show()
tuple(
    (p, c, idx2label[c])
    for p, c in zip(probs5[0][0].detach().numpy(), probs5[1][0].detach().numpy())
)

(tabby — это тоже кошка)

Применим LIME

In [ ]:
!pip -q install lime

Lime генерирует массив изображений из исходного входного изображения 

Таким образом, нам нужно предоставить конструктору:
1. Исходное изображение в виде массива numpy
2. Функцию классификации, которая будет принимать массив искаженных изображений в качестве входных данных и генерировать вероятности для каждого класса для каждого изображения в качестве выходных.



Поэтому потребуется вспомогательная функция для обработки пакета изображений, в соответствии с API LIME.

In [ ]:
import torch
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")


def batch_predict(images):  # images are numpy arrays
    model.eval()
    transform = get_input_transform()
    batch = torch.stack(tuple(transform(Image.fromarray(i)) for i in images), dim=0)

    model.to(device)
    batch = batch.to(device)

    logits = model(batch)
    probs = F.softmax(logits, dim=1)
    return probs.detach().cpu().numpy()

Создадим экзепляр ImageExplainer и сгенерируем объект explanation

In [ ]:
import lime
from lime import lime_image

explainer = lime_image.LimeImageExplainer(random_state=42)
explanation = explainer.explain_instance(
    np.array(img.resize((224, 224))),  # Lime assume that input is a numpy array :(
    batch_predict,  # classification function
    top_labels=5,
    hide_color=0,
    num_samples=1000,  # number of images that will be sent to classification function
    random_seed=42,
)

Выведем top5 предсказаний, сделанных через LIME.

P.S. Они не обязаны совпадать с предсказаниями для картинки без изменений.

In [ ]:
for i, id in enumerate(explanation.top_labels):
    print(i, idx2label[id])

Воспользуемся маской на изображении и посмотрим области, которые дают лучший прогноз.

In [ ]:
from skimage.segmentation import mark_boundaries

fig, ax = plt.subplots(nrows=1, ncols=2, figsize=(10, 10))

for i, id in enumerate(explanation.top_labels[:2]):
    temp, mask = explanation.get_image_and_mask(
        id, positive_only=False, num_features=5, hide_rest=False
    )
    img_boundry = mark_boundaries(temp, mask)
    ax[i].imshow(img_boundry)
    ax[i].set_title(idx2label[id])
    # number of clusters to be shown in the image: num_features=5
    # show or not negatively impacting clusters: positive_only=False
    # first 5 may be only positive

Зеленым цветом обозначена область наивысшего прогноза, оранжевым — области, которые меньше всего соответствуют нашему прогнозу. При `positive_only=False` будут показаны только границы.

И другое изображение:

In [ ]:
img_2 = get_image("cat_and_dog2.png")
plt.imshow(img_2)
plt.show()

Запуск Lime

In [ ]:
explainer = lime_image.LimeImageExplainer()
explanation = explainer.explain_instance(
    np.array(img_2.resize((224, 224))),
    batch_predict,  # classification function
    top_labels=5,
    hide_color=0,
    num_samples=1000,
)  # number of images that will be sent to classification function
# Display top labels
for i, id in enumerate(explanation.top_labels):
    print(i, idx2label[id])

Выведем сегменты, наиболее повлиявшие на каждое предсказание:

In [ ]:
fig, ax = plt.subplots(nrows=1, ncols=5, figsize=(50, 10))

for i, id in enumerate(explanation.top_labels):
    temp, mask = explanation.get_image_and_mask(
        id, positive_only=False, num_features=5, hide_rest=False
    )
    img_boundry = mark_boundaries(temp, mask)
    ax[i].imshow(img_boundry)
    ax[i].set_title(idx2label[id])

В то время как большинство методов, описанных в первом сообщении блога, являются глобальными методами, LIME — это метод, имеющий локальную область применения.

Таким образом, LIME позволяет нам объяснить конкретные прогнозы любого классификатора.

Структура LIME является гибкой в ​​том смысле, что любую интерпретируемую модель можно использовать для объяснения прогнозов.

Кроме того, концепция интерпретируемых компонентов позволяет применять LIME для объяснения данных большой размерности, таких как классификация изображений или текста.

## SHAP (SHapley Additive exPlanations)
[SHAP](https://christophm.github.io/interpretable-ml-book/shap.html)

###Принцип работы

<img src="https://edunet.kea.su/repo/EduNet-content/L10/out/shap_scheme.png" alt="alttext" width=600/>


Цель SHAP — объяснить предсказание объекта $x$ путем вычисления вклада каждого признака в предсказание. Для этого вычисляются SHAP значения, основанные на значениях Шепли из теории игр. 

SHAP рассматривает объясняемую модель как игру, а признаки, используемые в обучении, как коалицию игроков. SHAP значения говорят нам, как справедливо распределить "выигрыш" между игроками — это вклад, который каждый игрок вносит в предсказание модели. SHAP основывается на анализе локальной точности признаков, соответственно, "выигрыш" — это предсказание, сделанное моделью для одного образца. 

При этом игрок не обязательно должен быть индивидуальным признаком, он может состоять из группы признаков. Например, для моделей, работающих с изображениями, пиксели могут быть сгруппированы в суперпиксели, а "выигрыш" распределяется между ними. 




Итак, каким образом можно определить вклад признака в предсказание, сделанное моделью? Предположим у нас есть модель, которая предсказывает доход человека на основании его возраста, пола и профессии. Для определения вклада каждого признака рассмотрим все возможные комбинации $f$ признаков в модели ($f$ от $0$ до $3$) и представим их в виде графа:

<img src="https://edunet.kea.su/repo/EduNet-content/L10/out/shap_features_graph.png" alt="alttext" width=600/>

[SHAP Values Explained Exactly How You Wished Someone Explained to You](https://towardsdatascience.com/shap-explained-the-way-i-wish-someone-explained-it-to-me-ab81cc69ef30)

Здесь каждая вершина изображает набор признаков, а каждое ребро — добавление нового признака в набор.

Далее SHAP обучает модель на каждом наборе признаков в графе (сохраняя гиперпараметры модели и набор тренировочных объектов)

Предположим, что мы обучили модель на всех имеющихся наборах признаков и сделали предсказание зарплаты для объекта $x_0$.

Нулевая модель делает самое простое "предсказание" — усредняет зарплату, не учитывая никакие признаки (точность соответствующая).
Дальше мы можем использовать по одному признаку, более сложные модели базируются на нескольких признаках.

<img src="https://edunet.kea.su/repo/EduNet-content/L10/out/shap_predictions_graph.png" alt="alttext" width=550/>

Теперь в вершинах графа находятся предсказания, сделанные соответствующей моделью для объекта $x_0$. 

Зная предсказания всех возможных моделей для одного объекта, мы можем посчитать вклад каждого признака в предсказание. Вклад признака в предсказание высчитывается на основании его *маржинальных вкладов (marginal contribution)*. Маржинальный вклад признака &mdash; это разница между предсказанием модели, обученной на наборе, включающей данный признак, и модели, обученной на том же наборе, за исключением данного признака. В данном случае можно рассчитать  маржинальный вклад для признака как разницу предсказаний моделей, соединенных ребром в графе, а вес ребра и будет являться маржинальным вкладом.

Например, маржинальный вклад для признака Age в предсказание нулевой модели для объекта $x_0$ рассчитывается следующим образом:

$$MC_{Age,\{Age\}}(x_0)=Predict_{\{Age\}}(x_0)-Predict_{\emptyset}(x_0)=40k\,\$-50k\,\$=-10k\,\$$$

Для того, чтобы оценить вклад признака в предсказание модели, нужно учесть его маржинальные вклады во все модели, где этот признак присутствует (в графе выделены соответствующие ребра):

<img src="https://edunet.kea.su/repo/EduNet-content/L10/out/shap_estimation_important_features.png" alt="alttext" width=600/>

SHAP значение является общим вкладом признака в предсказание и вычисляется как взвешенная сумма маржинальных вкладов:
$$
\begin{multline*} 
SHAP_{Age}(x_0)=w_1\cdot MC_{Age,\{Age\}}(x_0)+w_2\cdot MC_{Age,\{Age, Gender\}}(x_0)\\+w_3\cdot MC_{Age,\{Age, Job\}}(x_0)+w_4\cdot MC_{Age,\{Age, Gender, Job\}}(x_0)
\end{multline*}
$$

Веса определяются согласно правилу: сумма весов маржинальных вкладов для каждого уровня графа (числа признаков в наборе $f$) должна быть равной 1

Таким образом, нетрудно рассчитать веса маржинальных вкладов признака Age: 
* Первый уровень содержит 3 ребра, каждое из которых будет иметь вес $\frac{1}{3}$
* Второй уровень содержит 6 ребер, каждое из которых будет иметь вес $\frac{1}{6}$
* Третий уровень содержит 3 ребра, каждое из которых будет иметь вес $\frac{1}{3}$

Таким образом: $\displaystyle w_1=w_4=\frac{1}{3}, \; w_2=w_3=\frac{1}{6}$.

В общем случае вес маржинального вклада в модель, обученную на наборе из $f$ признаков, обратно пропорционален числу маржинальных вкладов во все модели, обученные на наборе из $f$ признаков.

Число маржинальных вкладов во все модели, обученные на наборе из $f$ признаков, может быть рассчитано как: $f\cdot C_{F}^{f}$, где $F$ — количество признаков. Например, рассчитаем вес $w_2$ маржинального вклада $MC_{Age,\{Age, Gender\}}(x_0)$, согласно правилу: 

$$\displaystyle w_2=\left[f\cdot C_{F}^{f}\right]^{-1}=\left[2\cdot C_{3}^{2}\right]^{-1}=\frac{1}{6}$$

где $C_{F}^{f}$ — биномиальный коэффициент.

$$
\begin{pmatrix}
F\\
f
\end{pmatrix}=C_{F}^{f} = \frac{F!}{f!(F-f)!}$$


<img src="https://edunet.kea.su/repo/EduNet-content/L10/out/shap_compute_marginal_weights.png" alt="alttext" width=700/>

Такие SHAP значения можно легко посчитать для каждого признака для каждого объекта и, например, усреднить по объектам какой-то группы. 

Например, можно взять все объекты, которые модель оценивает высоко, и посмотреть значимые дли них признаки. Либо, если у нас есть какая-то особенная группа объектов, можем контролировать, что для них модель выучивает именно важные признаки, а не какой-то мусор.

### Пример для табличных данных (Boston Dataset)

Установим пакет SHAP 

In [ ]:
from IPython.display import clear_output

!pip install shap
clear_output()

Для примера скачаем датасет жилья Бостона (boston_dataset), в котором проанализируем зависимость цены на жилье от параметров жилья и района, в котором оно находится.

Разобьем данные на train и test и обучим Random Forest модель.

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
import pandas as pd
import numpy as np


# load dataset
boston_dataset = pd.read_csv(
    "https://edunet.kea.su/repo/EduNet-web_dependencies/L10/boston_dataset.csv",
    index_col=0,
)
x_data = boston_dataset.iloc[:, :-1]
y_data = boston_dataset["target"]

# Split the data into train and test data
x_train, x_test, y_train, y_test = train_test_split(
    x_data, y_data, test_size=0.2, random_state=42
)

# Build the model with the random forest regression algorithm
rng = np.random.RandomState(42)
model = RandomForestRegressor(n_jobs=-1, max_depth=4, random_state=rng)
model.fit(x_train, y_train)

Применим Shap.

In [ ]:
import shap

# explain the model's predictions using SHAP
# (same syntax works for LightGBM, CatBoost, scikit-learn and spark models)
explainer = shap.TreeExplainer(model)
shap_values = explainer.shap_values(x_test)

In [ ]:
print(type(shap_values))  # numpy.ndarray

**Force plots** 

Хороший способ визуализировать вклад каждого признака в конкретный прогноз — использовать график сил.

В приведенном ниже примере показан график силы для первого объекта в тестовом наборе данных.

In [ ]:
# load JS visualization code to notebook
shap.initjs()
# visualize the first prediction’s explanation
shap.force_plot(explainer.expected_value, shap_values[0, :], x_test.iloc[0, :])

* $f(x)$ — это прогноз модели по анализируемому объекту недвижимости. А base_value - это средний прогноз по всему набору тестовых данных. Или, другими словами, это значение, которое можно было бы спрогнозировать, если бы мы не знали никаких характеристик текущего примера.

* Элементы, которые способствуют увеличению цены, показаны красным, а те, которые уменьшают — синим.

**Waterfall_plot**

Другой способ понимания влияния факторов для конкретного примера:

In [ ]:
# visualize the first prediction's explanation using waterfall

shap.plots._waterfall.waterfall_legacy(
    explainer.expected_value[0], shap_values[0, :], x_test.iloc[0, :]
)

Этот график объясняет движущие силы конкретного прогноза:

Влияние каждого отдельного признака (наименее значимые признаки объединяются в одну группу) представлено стрелками, которые перемещают логарифмическое отношение шансов влево и вправо, начиная с базового значения.

---

Указанный выше пример приведен только для одного объекта.

Если мы возьмем много пояснений Force plots, повернем их на 90 градусов, а затем сложим их по горизонтали, мы сможем увидеть объяснения для всего набора данных (в notebook этот график является интерактивным):

In [ ]:
# load JS visualization code to notebook
shap.initjs()
# visualize the training set predictions
shap.force_plot(explainer.expected_value, shap_values, x_test)

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns

df = pd.DataFrame(shap_values, columns=x_test.columns)
sns.clustermap(df)
plt.show()

**Summary plot**

Сводный график с `plot_type = 'bar'` даст нам график важности признаков.

Признаки с высокой предсказательной способностью показаны вверху, а с низкой предсказательной силой — внизу.

In [ ]:
shap.summary_plot(shap_values, x_test, plot_type="bar")

Обратите внимание, что согласно Shap наименьшей предсказательной способностью обладают признаки **CHAS, ZN, RAD** и **INDUS**.

Здесь мы только что рассмотрели алгоритм TreeExplainer для интерпретации модели.

Вы можете изучить остальные алгоритмы: DeepExplainer, KernelExplainer, LinearExplainer и GradientExplainer.

### Пример NLP (перевод с английского на русский)
Рассмотрим пример интерпретации модели для предварительно обученной модели машинного перевода
[Machine Translation Example](https://github.com/slundberg/shap/blob/master/notebooks/text_examples/translation/Machine%20Translation%20Explanations.ipynb). И, раз уж мы будем для перевода использовать предобученную модель-трансформер, то переведем начало статьи [How Transformers Work](https://towardsdatascience.com/transformers-141e32e69591)

Используем одну из моделей [huggingface ](https://github.com/huggingface/transformers)


Модель: [Language Technology in Helsinki](https://blogs.helsinki.fi/language-technology/)


[Language Technology Research Group at the University of Helsinki](https://huggingface.co/Helsinki-NLP)


[Helsinki-NLP/opus-mt-en-ru](https://huggingface.co/Helsinki-NLP/opus-mt-en-ru)

In [ ]:
try:
    import transformers
except ModuleNotFoundError:
    !pip install sentencepiece
    !pip install transformers
    clear_output()
    import transformers

Загружаем модель.

Для этого используется класс-фабрика, на вход которому передается имя модели, а возвращает он объект соответствующего класса.

In [ ]:
import torch
from transformers import AutoModelForSeq2SeqLM

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

lang = "en"
target_lang = "ru"
model_name = f"Helsinki-NLP/opus-mt-{lang}-{target_lang}"

# Download the model and the tokenizer
# can also try translation with different pre-trained models

# It's a Factory pattern
model = AutoModelForSeq2SeqLM.from_pretrained(model_name)
model.to(device)
clear_output()
print(type(model))

В данном случае нам вернулся объект типа [MarianMT](https://huggingface.co/transformers/model_doc/marian.html)



Теперь создадим [токенайзер](https://huggingface.co/transformers/main_classes/tokenizer.html).

Как мы уже обсуждали, токенайзер преобразует слова и знаки препинания из исходного текста в токены, которые можно подать на вход модели. В данном случае возвращаются id. При этом не всегда одно слово преобразуется в один токен, иногда слово разбивается по слогам на несколько токенов.


Создается токенайзер так же фабрикой по имени модели.


In [ ]:
!pip -q install sacremoses

In [ ]:
from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained(model_name, max_new_tokens=512)
print(type(tokenizer))

input = tokenizer("Hello world!", return_tensors="pt")
print(input)

translated = model.generate(
    **tokenizer("Hello world!", return_tensors="pt").to(device), max_new_tokens=512
)
# ** -  is dictionary unpack operator
# https://towardsdatascience.com/unpacking-operators-in-python-306ae44cd480

Теперь переведем целую фразу и проанализируем, как выход модели связан со входом.

Для этого создадим объект [shap.Explainer](https://shap.readthedocs.io/en/latest/generated/shap.Explainer.html), который в данном случае инициализируется экземпляром модели и экземпляром токенайзера*.


Вместо того, чтобы запускать саму модель, мы запускаем `Explainer` (неявно вызывая его метод `__call__`). 


\* В действительности вторым параметром конструктора `shap.Explainer` не обязательно должен быть токенайзер. `shap.Explainer` принимает объект, поддерживающий интерфейс `masker`:

```masked_args = masker(*model_args, mask=mask)```

Он используется для исключения части аргументов, и токенайзеры поддерживают этот интерфейс (`shap.TokenMasker`). Благодаря такому подходу shap может работать с различными моделями как с "черным ящиком", 


In [ ]:
import shap

# define the input sentences we want to translate
data = [
    "Transformers are a type of neural network architecture that have been gaining popularity. Transformers were developed to solve the problem of sequence transduction, or neural machine translation. That means any task that transforms an input sequence to an output sequence. This includes speech recognition, text-to-speech transformation, etc.."
]

# we build an explainer by passing the model we want to explain and
# the tokenizer we want to use to break up the input strings
explainer = shap.Explainer(model, tokenizer)

# explainers are callable, just like models
explanation = explainer(data)
clear_output()

На выходе получаем объект класса [shap.Explanation](https://shap.readthedocs.io/en/latest/generated/shap.Explanation.html#shap-explanation), который содержит значения Шепли для каждого токена.

In [ ]:
print("Data", explanation.data)
print("Shap values", explanation.values)
print("Shape", explanation.shape)  # 1, in, out

Теперь, используя магию shap, можно визуализировать результат.

In [ ]:
shap.initjs()
shap.plots.text(explanation)

### Пример NLP (абстрактное обобщение текста)

[[text] plot](https://shap.readthedocs.io/en/latest/example_notebooks/api_examples/plots/text.html)

В этом примере интерпретируется модель генерации объяснений для предварительно обученной модели для составления краткого резюме статьи.

Используется датасет Extreme Summarization [XSum](https://huggingface.co/sshleifer/distilbart-xsum-12-6)

In [ ]:
from IPython.display import clear_output

!pip install datasets
!pip install transformers
clear_output()

In [ ]:
from datasets import load_dataset
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM
import torch

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

tokenizer = AutoTokenizer.from_pretrained("sshleifer/distilbart-xsum-12-6")
model = AutoModelForSeq2SeqLM.from_pretrained("sshleifer/distilbart-xsum-12-6").to(
    device
)


dataset = load_dataset("xsum", split="train")  # load dataset
s = dataset["document"][0:1]  # slice inputs from dataset to run model inference on
explainer = shap.Explainer(model, tokenizer)  # create an explainer object
explanation = explainer(s)  # Compute shap values
clear_output()

In [ ]:
shap.initjs()
shap.plots.text(explanation)  # Visualize shap explanations

### SHAP Deep Explainer



Существует несколько подходов к интерпретации результатов предсказания моделей: подходы, основанные на изменении данных, и градиентные методы. Однако использование этих методов может приводить к некорректным результатам при некоторых обстоятельствах.

Рассмотрим в качестве примера проблему насыщения (saturation):

<img src ="https://edunet.kea.su/repo/EduNet-web_dependencies/L10/saturation_problem.png" width="800">

<center><p><em>Source: <a href="https://arxiv.org/pdf/1704.02685.pdf">Learning Important Features Through Propagating Activation Differences</a></p></em></center>


Пусть $h$ — аналог активации некоторого нейрона, вычисляющийся как $h=max(0,1-i_1-i_2)$. Если мы возьмем значения признаков $i_1=1$ и $i_2=1$, то на выходе получим значение $h=0$. Далее по очереди будем занулять значения каждого из признаков, внося таким образом пертурбации: $i_1=0$ и $i_2=1$,  $i_1=1$ и $i_2=0$. В обоих случаях выход по-прежнему будет $h=0$. Может сложиться обманчивое впечатление, что ни один из признаков не влияет на результат вычисления. Таким образом, мы столкнулись с проблемой, что подход, основанный на изменении признаков, будет занижать значимость признаков, чей вклад в результат достиг насыщения. Аналогично градиентные методы также будут недооценивать важность признаков при насыщении, поскольку градиент в данном случае будет равным 0.

Проблема насыщения не является редкой, в частности, с ней можно столкнуться в биологии при построении [моделей](https://www.nature.com/articles/nmeth.3547), объясняющих вклад единичных мутаций на то или иное свойство организма, что связано с вырожденностью генетического кода.

Альтернативный способ объяснения предсказаний моделей использует алгоритм [DeepLIFT](https://arxiv.org/abs/1704.02685), который базируется на идее GradientAscending, но не ограничивается ей. Основная идея DeepLIFT заключается в том, что он оценивает важность признака с точки зрения отличий от некоторого «референса», где референс выбирается в соответствии с решаемой проблемой. Референс для входных данных представляет собой некий нейтральный объект, у которого отсутствует специфическое свойство, например, таким свойством может быть присутствие/отсутствие того или иного объекта на изображении. Помимо референса для входных данных, определяется референс для каждого нейрона (активация соответствующего нейрона, рассчитанная для референсного объекта), аналогично референсом выхода сети будет вычисленный выход для референсного объекта.

DeepLIFT объясняет разницу между выходом сети для целевого объекта и выходом сети для референсного объекта на основании разницы между этими объектами. Пусть $t$ представляет выход некоторого нейрона/сети, а $x_1,x_2,...,x_n$ — нейроны одного из предшествующих слоев или множество слоев, необходимые для расчета $t$. Пусть $t_0$ — референсная активация для $t$. Тогда мы можем определить разницу выхода нейрона/сети для целевого объекта с выходом для референсного объекта как $\Delta t=t-t_0$. Тогда мы можем разложить ее на вклады $C_{\Delta x_i \Delta t}$  разниц между активаций нейронов в предыдущих слоях для целевого объекта и референса ($\Delta x_i$):

$$\sum_{i=1}^nC_{\Delta x_i \Delta t}=\Delta t$$

Как именно DeepLIFT вычисляет эти вклады останется за рамками рассмотрения этой лекции. 

Использование такого подхода позволяет DeepLIFT решать проблему насыщения, приведенную на предыдущем рисунке, а также другую проблему, при которой градиент может совершать спонтанные скачки. Для иллюстрации этого примера рассмотрим функцию ReLU со сдвигом на $-10$: $y=max(0,x-10)$. Для этой функции градиент и вход, умноженный на градиент (этот подход также может быть использован для объяснения предсказаний нейронных сетей), имеют разрыв в точке $x=10$. И в отличие от этого подход, основанный на разнице с референсом, дает непрерывную величину оценки вклада признака.

<img src ="https://edunet.kea.su/repo/EduNet-web_dependencies/L10/deeplift_relu.png" width="800">

<center><p><em>Source: <a href="https://arxiv.org/pdf/1704.02685.pdf">Learning Important Features Through Propagating Activation Differences</a></p> </em></center>

###Пример изображения (ResNet18)

В качестве референса конкретно в SHAP DeepExlainer, построенном на основе DeepLIFT, используется не один объект, а усреднение набора произвольных изображений из датасета ImageNet (`shap.datasets.imagenet50`).

Попробуем воспользоваться этим на практике:

In [ ]:
!wget -q 'https://edunet.kea.su/repo/EduNet-web_dependencies/L10/cat_and_dog1.jpg' -O cat_and_dog1.jpg
!wget -q 'https://edunet.kea.su/repo/EduNet-web_dependencies/L10/cat_and_dog2.png' -O cat_and_dog2.png
!wget -q 'https://edunet.kea.su/repo/EduNet-web_dependencies/L10/imagenet_class_index.json' -O imagenet_class_index.json

In [ ]:
import os
import matplotlib.pyplot as plt
from PIL import Image


def get_image(path):
    with open(os.path.abspath(path), "rb") as f:
        with Image.open(f) as img:
            return img.convert("RGB")


img = get_image("cat_and_dog1.jpg")
plt.imshow(img)
plt.show()

In [ ]:
from torchvision import transforms


# resize & normalize
def get_input_transform():
    normalize = transforms.Normalize(
        mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]
    )
    transform = transforms.Compose(
        [
            transforms.Resize((256, 256)),
            transforms.CenterCrop((224, 224)),
            transforms.ToTensor(),
            normalize,
        ]
    )
    return transform


def get_input_tensors(img):
    transform = get_input_transform()
    # unsqeeze converts single image to batch of 1
    return transform(img).unsqueeze(0)

In [ ]:
from torchvision import models
import json

model = models.resnet18(weights="ResNet18_Weights.DEFAULT")

idx2label, cls2label, cls2idx = [], {}, {}
with open(os.path.abspath("/content/imagenet_class_index.json"), "r") as read_file:
    class_idx = json.load(read_file)
    idx2label = [class_idx[str(k)][1] for k in range(len(class_idx))]
    cls2label = {
        class_idx[str(k)][0]: class_idx[str(k)][1] for k in range(len(class_idx))
    }
    cls2idx = {class_idx[str(k)][0]: k for k in range(len(class_idx))}

In [ ]:
import torch.nn.functional as F

img_t = get_input_tensors(img)
model.eval()
model.cpu()
logits = model(img_t)

probs = F.softmax(logits, dim=1)
probs5 = probs.topk(5)
fig = plt.figure(figsize=(5, 5))
plt.imshow(np.array(img.resize((256, 256)).crop((16, 16, 240, 240))))
plt.show()
tuple(
    (p, c, idx2label[c])
    for p, c in zip(probs5[0][0].detach().numpy(), probs5[1][0].detach().numpy())
)

In [ ]:
!pip -q install shap

Посмотрим на изображения, выбранные в качестве референса

In [ ]:
imagenet_50, broken_targets = shap.datasets.imagenet50()

print("Data shape", imagenet_50.shape, type(imagenet_50))
# Show first image
fig = plt.figure(figsize=(8, 8))
plt.imshow(imagenet_50[0].astype("int"))
plt.show()

Можно взглянуть и на остальные картинки

In [ ]:
fig, ax = plt.subplots(nrows=5, ncols=10, figsize=(25, 10))
for i, imgs in enumerate(imagenet_50):
    row = i // 5
    col = i % 5
    ax[col, row].imshow(imgs.astype("int"))

In [ ]:
# for performance reason use as background only 10 images in PyTorch format
background = torch.tensor(imagenet_50[0:10]).permute(0, 3, 1, 2).to(device) / 255

# https://shap-lrjball.readthedocs.io/en/latest/generated/shap.DeepExplainer.html
explainer = shap.DeepExplainer(model.to(device), background)
shap_values = explainer.shap_values(img_t)  # List
clear_output()

По умолчанию возвращаются shap индексы для каждого класса, для каждого пикселя 

In [ ]:
print("Classes", len(shap_values))
print("Values", shap_values[0].shape)

Для отображения результатов используем метод [shap.image_plot](https://shap-lrjball.readthedocs.io/en/latest/generated/shap.image_plot.html?highlight=image_plot)

его API ждет данные в виде списков numpy-массивов, поэтому нам потребуется преобразовать данные.


In [ ]:
img = get_image("cat_and_dog1.jpg")
# Get indexes of top5 classes predicted by model
top5_indexes = probs5.indices.squeeze(0).numpy().astype("int")

# Get shap values for this classes
shap_values_for_top_results = np.array(shap_values)[top5_indexes]

# Move color channels back for numpy compability (...,3,224,224 ) - > (...,224,224,3)
shap_values_for_top_results = np.swapaxes(
    shap_values_for_top_results, 4, 2
)  # swapaxes is do the same thing as torch.permute

# Convert first dim of numpy.array back to pythol list as required shap.image_plot api
shap_values_for_top_results = list(shap_values_for_top_results[:])

# Prepare test image
test_image = (
    np.array(img.resize((224, 224))).reshape((1, 224, 224, 3)) / 255
)  # resize to size of shap values
# test_image = test_image/255 # [np.newaxis, ...]


# Get labels for top5 classes

shap_labels = np.array(idx2label)[top5_indexes]
shap_labels = [list(shap_labels)]  # One list for sample

print(
    "Len of shap_values list", len(shap_values_for_top_results)
)  # number of classes to explain
print("Shape of one value", shap_values_for_top_results[0].shape)  # n_samples, H,W,C
print(shap_labels)  # n_samples, number of classes

Теперь визуализируем результаты:

In [ ]:
shap.image_plot(shap_values_for_top_results, test_image, labels=shap_labels)

На практике чаще всего интересны объяснения только для нескольких классов с максимальной уверенностью.

И в документации описан параметр, который позволяет возвращать объяснения только для этих классов.

[[doc] shap.DeepExplainer](https://shap-lrjball.readthedocs.io/en/latest/generated/shap.DeepExplainer.html)

`ranked_outputs = 5, output_rank_order ='max'`

При этом возвращается кортеж:
shap_values и индексы классов, для которых получено объяснение.



In [ ]:
shape_values_for_best_pred, indexes = explainer.shap_values(
    img_t, ranked_outputs=5, output_rank_order="max"
)  # List

Результаты снова надо преобразовать из PyTorch формата:

In [ ]:
# Convert shap val from PyTorch format
shape_values_for_best_pred = np.swapaxes(np.array(shape_values_for_best_pred), 4, 2)
shape_values_for_best_pred = list(shape_values_for_best_pred[:])

# Get names for returned indexes
shap_labels = np.array(idx2label)[indexes.cpu()[0]]
shap_labels = [list(shap_labels)]  # One list for sample

Визуализируем результат:

In [ ]:
shap.image_plot(shape_values_for_best_pred, test_image, labels=shap_labels)

## Другие библиотеки

[[git] DEN](https://github.com/isaacrob/DEN) : основан на новом режиме обучения сиамской нейронной сети без учителя и функции потерь, который называется Differentiating EmbeddingNetworks (DEN).

Сиамская нейронная сеть находит отличительные или похожие черты между конкретными парами образцов в наборе данных и использует эти функции, чтобы встроить набор данных в пространство более низкой размерности, где он может быть визуализирован.

В отличие от существующих алгоритмов визуализации, таких как UMAP ort-SNE, DEN является параметрическим, то есть его можно интерпретировать такими методами, как SHAP.

[[towardsdatascience] XAI](https://towardsdatascience.com/xai-build-your-own-deep-learning-interpretation-algorithm-6e471b59af7) 

Зачем беспокоиться о выборе всех необходимых признаков?

Когда вы пытаетесь понять феномен, создавший ваши данные, вы должны заботиться о тех факторах, которые способствуют этому, а не только о самых грубых признаках этого в контексте вашей методологии. Но помните, что минимальный оптимальный набор признаков по определению зависит от вашего выбора классификатора. В примере ниже можно увеличить число слоев, и отвергнутые ранее признаки окажутся востребованными моделью.

[[towardsdatascience] Boruta Feature Selection](https://towardsdatascience.com/simple-example-using-boruta-feature-selection-in-python-8b96925d5d7a) 

# Градиентные методы оценки важности признаков

## Vanilla Gradient

### Идея метода 

Имея информацию о структуре модели, можно использовать градиент, чтобы понять, как входы связаны с выходом.

Градиент указывает направление возрастания функции. Если мы выберем  **logit, соответствующий метке наиболее вероятного класса**, и посчитаем для него **градиент по исходному изображению**, мы можем узнать, какие пиксели нужно “усилить”, чтобы модель была более уверена в ответе.


<center><img src ="https://edunet.kea.su/repo/EduNet-web_dependencies/L10/saliency_via_backprop.png" width="800">

<p><em>Source: <a href="https://arxiv.org/pdf/1312.6034.pdf">Deep Inside Convolutional Networks: Visualising Image Classification Models and Saliency Maps
</a></p> </em></center>

Тут важно заметить, что пример из статьи на картинке выше - это **cherry picked** (специально отобранный пример, для которого данный метод дает хороший результат). Давайте посмотрим на нашей картинке.

###Пример изображения (ResNet18)

Загрузим изображение

In [ ]:
!wget -q 'https://edunet.kea.su/repo/EduNet-web_dependencies/L10/cat_and_dog1.jpg' -O cat_and_dog1.jpg
!wget -q 'https://edunet.kea.su/repo/EduNet-web_dependencies/L10/cat_and_dog2.png' -O cat_and_dog2.png
!wget -q 'https://edunet.kea.su/repo/EduNet-web_dependencies/L10/imagenet_class_index.json' -O imagenet_class_index.json

In [ ]:
import os
import matplotlib.pyplot as plt
from PIL import Image


def get_image(path):
    with open(os.path.abspath(path), "rb") as f:
        with Image.open(f) as img:
            return img.convert("RGB")


img = get_image("cat_and_dog1.jpg")
plt.rcParams["figure.figsize"] = (5, 5)
plt.imshow(img)
plt.show()

Теперь нам нужно преобразовать это изображение в тензор PyTorch, а также его нормализовать для использования в нашей предварительно обученной модели.

In [ ]:
from torchvision import transforms

# resize & normalize
def get_input_transform():
    transform = transforms.Compose([
        transforms.Resize(224),
        transforms.CenterCrop((224, 224)),
        transforms.ToTensor(),
        transforms.Normalize(
            mean=(0.485, 0.456, 0.406), 
            std=(0.229, 0.224, 0.225),
        )
    ])
    return transform

# for get croped img from imput tensor
def get_reverse_transform():
    transform = transforms.Compose([
        transforms.Normalize(
            mean=(0., 0., 0.), 
            std=(1/0.229, 1/0.224, 1/0.225)
        ), 
        transforms.Normalize(
            mean=(-0.485, -0.456, -0.406),
            std=(1., 1., 1.),
        ),
        transforms.Lambda(
            lambda x: torch.permute(x, (0, 2, 3, 1))
        ),
        transforms.Lambda(
            lambda x: x.detach().numpy()
        ),
    ])
    return transform
       

def get_input_tensors(img):
    transform = get_input_transform()
    # unsqeeze converts single image to batch of 1
    return transform(img).unsqueeze(0)

def get_crop_img(img_tensor):
    transform = get_reverse_transform()
    return transform(img_tensor)[0]


Загрузим предобученную модель ResNet18, доступную в PyTorch, и классы изображений из ImageNet.

In [ ]:
import torch
from torchvision import models
import json

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

model = models.resnet18(weights="ResNet18_Weights.DEFAULT")

idx2label, cls2label, cls2idx = [], {}, {}
with open(os.path.abspath("/content/imagenet_class_index.json"), "r") as read_file:
    class_idx = json.load(read_file)
    idx2label = [class_idx[str(k)][1] for k in range(len(class_idx))]
    cls2label = {
        class_idx[str(k)][0]: class_idx[str(k)][1] for k in range(len(class_idx))
    }
    cls2idx = {class_idx[str(k)][0]: k for k in range(len(class_idx))}

Получим предсказание. А после этого полученные нами прогнозы (logits) пропустим через softmax, чтобы получить вероятности и метки классов для 6 лучших прогнозов.

In [ ]:
import torch.nn.functional as F
print(type(img))
img_t = get_input_tensors(img)

model.to(device)
model.eval()
logits = model(img_t.to(device))

In [ ]:
def top_k_class(logits, k=6):
    prediction = F.softmax(logits, dim=1)
    top_props, top_inds = prediction.topk(k)

    for i in range(k):
        category_name = idx2label[top_inds[0][i].item()]
        score = top_props[0][i].item()
        print(f"{category_name} {top_inds[0][i].item()}: {100 * score:.1f}%")

top_k_class(logits)

Включаем расчет градиента для изображения. Делаем предсказание. Выбираем наиболее вероятный logit.

In [ ]:
img_t.requires_grad = True  # Tell pytorch to compute grads w.r.t. inputs too

logits = model(img_t.to(device))  # [1,1000] batch of one element, 1000 class scores
top_score, top_idx = logits[0].topk(1)  # Get id of class with best score
id = top_idx[0].item()

print(id, idx2label[id])  # Print the label this class

score = logits[:, id]  # Model output for particular class

Для выхода модели, соответствующего нашему классу, рассчитываем градиент.

In [ ]:
# Compute gradients
score.backward(retain_graph=True)

# retain_grad = True is not nessesary
# But if we run this code second time, we got a torch error without it
# because pytorch want to accumulate gradients explicitly

print(img_t.grad.shape)

У входного тензора (изображения) появился градиент, который указывает на то, как каждый элемент повлиял на выход модели. Отобразим этот градиент.

In [ ]:
import numpy as np
from matplotlib import pylab as P


# Helper method to display grad
def grad_to_image(raw_grads, percentile=99):
    gradients = raw_grads.detach().numpy()
    gradients = np.transpose(gradients, (0, 2, 3, 1))[0]
    
    image_2d = np.sum(np.abs(gradients), axis=2)

    vmax = np.percentile(image_2d, percentile)
    vmin = np.min(image_2d)

    return np.clip((image_2d - vmin) / (vmax - vmin), 0, 1)


def plot_saliency_map(img_tensor, saliency_map):
    plt.rcParams["figure.figsize"] = (10, 5)
    plt.subplot(1, 2, 1)
    img = get_crop_img(img_t)
    plt.imshow(img)
    plt.axis('off')
    plt.subplot(1, 2, 2)
    plt.imshow(saliency_map, cmap=P.cm.gray, vmin=0, vmax=1)
    plt.axis('off')
    plt.show()

In [ ]:
saliency_map = grad_to_image(img_t.grad)
plot_saliency_map(img_t, saliency_map)

**Карта важности (saliency map)**, полученная таким образом, получается очень зашумленной. 

### Adversarial attacks

Принцип взятия градиента по входу используется при **состязательных атаках (adversarial attacks)**.

Если не просто визуализировать градиент, а с его помощью менять изображение, усиливая определенную метку класса, то можно обмануть сеть и заставить ее неверно классифицировать картинку, незначительно поменяв ее.

<img src ="https://edunet.kea.su/repo/EduNet-web_dependencies/L10/adversarial_attack.jpg" width="700">




Подробнее:
[[wiki] Adversarial machine learning](https://en.wikipedia.org/wiki/Adversarial_machine_learning)

[The Intuition behind Adversarial Attacks on Neural Networks](https://blog.mlreview.com/the-intuition-behind-adversarial-attacks-on-neural-networks-71fdd427a33b)


## SmoothGrad


### Идея метода
Проблемой Vanilla Gradient Ascent является большая зашумленность карты важности. Было придумано несколько способов борьбы с этим, один из них был предложен в статье [SmoothGrad: removing noise by adding noise](https://arxiv.org/pdf/1706.03825.pdf). 

Как вы можете догадаться из названия статьи - идея **SmoothGrad** заключается в **добавлении** к исходному изображению $x$ **гауссового шума**:

$$x+\mathcal{N}(0, \sigma^2).$$

Для набора зашумленных изображений с помощью **Vanilla Gradient** рассчитываются **карты важности (saliency map)**:

$$M_c(x+\mathcal{N}(0, \sigma^2)).$$

**Карты важности**, полученные от зашумленных изображений, **усредняются**:

$$SmoothGrad = \frac{1}{n}\sum_{1}^{n}M_c(x+\mathcal{N}(0, \sigma^2)),$$

$SmoothGrad$ - результат SmoothGrad.






<center><img src ="https://edunet.kea.su/repo/EduNet-web_dependencies/L10/smoth_grad.png" width="800">
<p><em>Source: <a href="https://arxiv.org/pdf/1706.03825.pdf">SmoothGrad: removing noise by adding noise
</a></p> </em></center>

В статье рекомендуют выбирать $n = 50$ и 10-20% шума. Уровень шума определяют как отношение:

$$\frac{\sigma}{x_{max}-x_{min}}.$$



###Пример изображения (ResNet18)

Для визуализации работы SmoothGrad используем [код](https://github.com/PAIR-code/saliency/blob/master/Examples_pytorch.ipynb) c [сайта](https://pair-code.github.io/saliency/), посвященного [статье](https://arxiv.org/pdf/1706.03825.pdf).

In [ ]:
! pip install saliency

У кода есть особенность: нужно написать функцию `call_model_function`, вызывающую модель. 

При этом любое изображение поданное в метод `GetMask` класса `GradientSaliency` будет преобразовано в `np.array` к тому же размер входного изображения и карты важности на выходе должен совпадать, что осложняет использование `torchvision.transforms`.

In [ ]:
import saliency.core as saliency
import numpy as np

from PIL import Image


model = models.resnet18(weights="ResNet18_Weights.DEFAULT")

def call_model_function(img, call_model_args=None, expected_keys=None):
    img_t = torch.tensor(np.transpose(img, (0, 3, 1, 2)))
    transform = transforms.Normalize(
        mean=(0.485, 0.456, 0.406), 
        std=(0.229, 0.224, 0.225),
    )
    img_t = transform(img_t)
    img_t.requires_grad_(True)

    model.to(device)
    model.eval()
    logits = model(img_t.float().to(device))

    top_score, top_idx = logits[0].topk(1)  # Get id of class with best score
    target_class_idx = top_idx[0].item()

    output = logits[:, target_class_idx]
    grads = torch.autograd.grad(
        output,
        img_t,
        grad_outputs=torch.ones_like(output)
    ) # output[:, target_class_idx]
    grads = torch.movedim(grads[0], 1, 3)
    gradients = grads.detach().numpy()
    return {saliency.base.INPUT_OUTPUT_GRADIENTS: gradients}

Получаем Vanilla Gradient, рассчитанный для исходного изображения с помощью установленной библиотеки и результат  SmoothGrad.

In [ ]:
img = get_image("cat_and_dog1.jpg")
img_t = get_input_tensors(img)
img_arr = get_crop_img(img_t)

gradient_saliency = saliency.GradientSaliency()

vanilla_mask_3d = gradient_saliency.GetMask(img_arr, call_model_function)
smoothgrad_mask_3d = gradient_saliency.GetSmoothedMask(img_arr, 
                                                       call_model_function)

# Call the visualization methods to convert the 3D tensors to 2D grayscale.
vanilla_mask_grayscale = saliency.VisualizeImageGrayscale(vanilla_mask_3d)
smoothgrad_mask_grayscale = saliency.VisualizeImageGrayscale(smoothgrad_mask_3d)

Визуализируем результат:

In [ ]:
from matplotlib import pylab as P

def ShowGrayscaleImage(im, title='', ax=None):
    if ax is None:
        P.figure()
    P.axis('off')
    P.imshow(im, cmap=P.cm.gray, vmin=0, vmax=1)
    P.title(title)

# Set up matplot lib figures.
plt.rcParams["figure.figsize"] = (15, 5)

plt.subplot(1, 3, 1)
plt.imshow(img_arr)
plt.axis('off')

ShowGrayscaleImage(
    vanilla_mask_grayscale, 
    title='Vanilla Gradient', 
    ax=P.subplot(1, 3, 2)
)
ShowGrayscaleImage(
    smoothgrad_mask_grayscale,
    title='SmoothGrad',
    ax=P.subplot(1, 3, 3)
)

##Integrated Gradients

### Идея метода

Следующий метод, который мы посмотрим, называется **Integrated Gradients**. Он напоминает **SmoothGrad** тем, что мы намеренно "портим" изображения. Давайте разберемся, как он работает. 


В методе **Integrated Gradients** мы вибираем **опроное изображение** $x'$. В качестве опорного изображения используются черный фон (все нули по RGB каналам). Оцениваемое изображение $x$ примешивают к опорному изображению $x’$ с пропорцией $\alpha$:

$$x'+\alpha(x-x')$$

Таким образом мы портим изображение и постепенно его восстанавливаем.

<center><img src ="https://edunet.kea.su/repo/EduNet-web_dependencies/L10/IG_fireboat_1.png" width="900">

<p><em>Source: <a href="https://www.tensorflow.org/tutorials/interpretability/integrated_gradients">Tensorflow tutorials: Integrated gradients
</a></p> </em></center>

Для смеси изображений считается **Vanilla Gradient**: 

$$M_c(x'+\alpha(x-x'))$$

Формула, лежащая в основе **Integrated Gradients** была предложена в [статье](https://arxiv.org/pdf/1703.01365.pdf). Это - интегральное значение градиента при восстановлении изображения.

$$IntegratedGrads(x) = (x-x')\cdot\int_{\alpha=0}^1 M_c(x'+\alpha(x-x'))dα$$

Множитель $x-x’$ появился т.к. изначально градиент был по $dx = (x-x’)d\alpha$.

В расчетах интеграл аппроксимируется суммой:


$$IntegratedGrads(x) \approx (x-x')\cdot\sum_{k=1}^m M_c(x'+\frac{k}{m}(x-x'))\cdot\frac{1}{m}$$

Значение $m$ выбирают в диапазоне от 20 до 300.

Пример результата:
<center><img src ="https://edunet.kea.su/repo/EduNet-web_dependencies/L10/IG_fireboat.png" width="500">

<p><em>Source: <a href="https://www.tensorflow.org/tutorials/interpretability/integrated_gradients">Tensorflow tutorials: Integrated gradients
</a></p> </em></center>

###Пример изображения (ResNet18)

In [ ]:
! pip install captum

In [ ]:
output = model(img_t)
output = F.softmax(output, dim=1)
prediction_score, pred_label_idx = torch.topk(output, 1)

pred_label_idx.squeeze_()
predicted_label = idx2label[pred_label_idx.item()]
print('Predicted:', predicted_label, '(', prediction_score.squeeze().item(), ')')

In [ ]:
from captum.attr import IntegratedGradients


integrated_gradients = IntegratedGradients(model)
attributions_ig = integrated_gradients.attribute(
    img_t,
    target=pred_label_idx,
    n_steps=200
)

In [ ]:
saliency_map = grad_to_image(attributions_ig)

# Set up matplot lib figures.
plt.rcParams["figure.figsize"] = (20, 5)

plt.subplot(1, 4, 1)
plt.imshow(img_arr)
plt.axis('off')

ShowGrayscaleImage(
    vanilla_mask_grayscale, 
    title='Vanilla Gradient', 
    ax=P.subplot(1, 4, 2)
)
ShowGrayscaleImage(
    smoothgrad_mask_grayscale,
    title='SmoothGrad',
    ax=P.subplot(1, 4, 3)
)
ShowGrayscaleImage(
    saliency_map,
    title='Integrated Gradients',
    ax=P.subplot(1, 4, 4)
)

Пакет [captum](https://captum.ai/docs/attribution_algorithms) можно использовать и для других модальностей данных, например для [NLP BERT](https://captum.ai/tutorials/Bert_SQUAD_Interpret2). Там реализовано большое количество модификаций алгоритма Integrated Gradients и не только.

## Grad-CAM

Развитием **Gradient Ascent** для сверточных нейронных сетей **CNN**, является метод **Grad-CAM** (Class activation maps).

#### Идея метода

После каждого **сверточного слоя** нейронной сети мы получаем **карты признаков**, сохраняющие информацию о **расположении объектов** на исходном изображении. При этом все значения признаков для одного канала получаются одним и тем же преобразованием исходного изображения (получаются применением одинаковых сверток с одинаковыми весами), то есть **один канал** — это **карта одного признака**. 

 Посмотрим на карты признаков для **ResNet18**. Для этого загрузим модель вместе с весами.

In [ ]:
model = models.resnet18(weights="ResNet18_Weights.DEFAULT")
model

Нам интересны сложные признаки, которые выделяются на последних сверточных слоях. ResNet18 был обучен на ImageNet с размерами входного изображения 224x224. Посмотрим на размеры на выходе последнего сверточного слоя. 

In [ ]:
from torchsummary import summary

summary(model.to("cpu"), (3, 224, 224), device="cpu")

На последнем сверточном слое получаем 512 каналов 7x7. Для их сохранения напишем hook, в котором будем сохранять значения активации на выходе модели. На этот раз оформим hook через класс. 


In [ ]:
from collections import defaultdict


def get_forward_hook(history_dict, key):
    def forward_hook(self, input_, output):
        history_dict[key] = output.detach().clone()

    return forward_hook

def register_model_hooks(model):
    hooks_data_history = defaultdict(list)
    forward_hook = get_forward_hook(hooks_data_history, "feature_map")
    model._modules["layer4"].register_forward_hook(forward_hook)
    return hooks_data_history

Предобработаем картинку: приведем к размеру 224x224 и нормализуем изображение в соответствии со статистикой ImageNet.

In [ ]:
img_t = get_input_tensors(img)

Пропустим картинку через сеть и сохраним значения активаций. 

In [ ]:
model = model.eval()
history = register_model_hooks(model) 
output = model(img_t)

print(history["feature_map"].shape)

Нарисуем первые 6 карт признаков. Чтобы растянуть карты по размеру изображения, используем `extent` и `interpolation='bilinear'`.

In [ ]:
fif, axs = plt.subplots(2, 3, figsize=(15, 10))

axs = axs.ravel()
for i in range(6):
    axs[i].imshow(img_arr)
    axs[i].imshow(
        history["feature_map"][0][i],
        alpha=0.6,
        extent=(0, 224, 224, 0),
        interpolation="bilinear",
        cmap="jet",
    )
    axs[i].axis("off");

Вспомним предсказание модели. 

In [ ]:
import torch.nn.functional as F

number_of_top_classes = 6

prediction = F.softmax(output, dim=1)
top_props, top_inds = prediction.topk(number_of_top_classes)


for i in range(number_of_top_classes):
    category_name = idx2label[top_inds[0][i].item()]
    score = top_props[0][i].item()
    print(f"{category_name} {top_inds[0][i].item()}: {100 * score:.1f}%")

Мы смогли понять, к какой части изображения относятся те или иные признаки. Теперь попробуем понять, как они соотносятся с классом. Мы помним, что градиент указывает направление возрастания функции. Если мы выберем логит, соответствующий метке класса, и посчитаем для него градиент, то мы сможем увидеть, какие признаки имеют положительные значения (при их увеличении модель будет больше уверена в оценке). 

Напишем хук для сохранения значения градиента. Мы смотрим значения градиента на выходе слоя перед `AdaptiveAvgPool2d`, поэтому сохраним только среднее значения (значения градиента для карт признаков одного канала будет одинаковым).

In [ ]:
def get_backward_hook(history_dict, key):
    def backward_hook(self, grad_input_, grad_output):  # for tensors
        history_dict[key] = grad_output[0].detach().clone().mean(
            dim=[2, 3],
            keepdim=True
        )
    return backward_hook


def register_model_hooks(model):
    hooks_data_history = defaultdict(list)

    forward_hook = get_forward_hook(hooks_data_history, "feature_map")
    model._modules["layer4"].register_forward_hook(forward_hook)

    backward_hook = get_backward_hook(hooks_data_history, "weight")
    model._modules["layer4"].register_full_backward_hook(backward_hook)
    return hooks_data_history

Итоговая формула **Grad-CAM** (Class activation maps): 

$$CAM = ReLU(\sum_{i=1}^{Nch}w_iA_i)$$

где $A_i$ — каналы карты признаков, $w_i$ — веса, полученные пропусканием градиента по логиту, соответствующему метке класса. $ReLU$ используется потому, что нам интересны только положительно влияющие на метку класса признаки. 

Функция, рассчитывающая CAM

In [ ]:
def get_cam_map(model, img, class_num):
    history = register_model_hooks(model)

    output = model.eval()(img)
    activation = history["feature_map"]

    output[0, class_num].backward()
    weight = history["weight"]

    cam_map = F.relu((weight[0] * activation[0]).sum(0)).detach().cpu()
    return cam_map

Визуализация важности признаков для top6 классов.

In [ ]:
model = models.resnet18(weights="ResNet18_Weights.DEFAULT")
fif, axs = plt.subplots(2, 3, figsize=(15, 10))

axs = axs.ravel()
for i in range(6):
    cam_map = get_cam_map(model, img_t, top_inds[0][i])

    axs[i].imshow(img_arr)
    axs[i].imshow(
        cam_map,
        alpha=0.6,
        extent=(0, 224, 224, 0),
        interpolation="bilinear",
        cmap="jet",
    )
    axs[i].set_title(idx2label[top_inds[0][i].item()])
    axs[i].axis("off");

###Пример изображения (ResNet18)

Можно сделать все то же самое с помощью библиотеки  Grad-CAM.

In [ ]:
from IPython.display import clear_output

!pip install grad-cam
clear_output()

In [ ]:
from pytorch_grad_cam import GradCAM

target_layers = [model._modules["layer4"]]

cam = GradCAM(model=model, target_layers=target_layers)

In [ ]:
from pytorch_grad_cam.utils.model_targets import ClassifierOutputTarget

fif, axs = plt.subplots(2, 3, figsize=(15, 10))

axs = axs.ravel()
for i in range(6):
    target = [ClassifierOutputTarget(top_inds[0][i])]
    cam_map = cam(input_tensor=img_t, targets=target)

    axs[i].imshow(img_arr)
    axs[i].imshow(cam_map[0], alpha=0.6, interpolation="bilinear", cmap="jet")
    axs[i].set_title(idx2label[top_inds[0][i].item()])
    axs[i].axis("off");

В библиотеке также реализованы другие методы визуализации, например, Grad-CAM++, использующий градиенты второго порядка. 

In [ ]:
from pytorch_grad_cam import GradCAMPlusPlus

target_layers = [model._modules["layer4"]]
cam_plus = GradCAMPlusPlus(model=model, target_layers=target_layers)

In [ ]:
fif, axs = plt.subplots(2, 3, figsize=(15, 10))

axs = axs.ravel()
for i in range(6):
    target = [ClassifierOutputTarget(top_inds[0][i])]
    cam_map = cam_plus(input_tensor=img_t, targets=target)

    axs[i].imshow(img_arr)
    axs[i].imshow(cam_map[0], alpha=0.6, interpolation="bilinear", cmap="jet")
    axs[i].set_title(idx2label[top_inds[0][i].item()])
    axs[i].axis("off");

Grad-CAM++ для ряда изображений работает лучше Grad-CAM.

<center><img src="https://edunet.kea.su/repo/EduNet-web_dependencies/L10/Grad_CAM_plus_plus.png" alt="alttext" width=600/></center>

<center><p><em>Source: <a href="https://arxiv.org/pdf/1710.11063.pdf">Grad-CAM++: Improved Visual Explanations for
Deep Convolutional Networks</a></p> </em></center>

Подробнее о других методах читайте в [документации grad-cam](https://jacobgil.github.io/pytorch-gradcam-book/introduction.html).

## Критика градиентных методов

Градиентные методы имеют существенное преимущество перед SHAP и LIME: они вычисляются быстрее, чем методы не зависящие от моделей. Но у них есть ряд недостатков.

Хорошим примером проблемы визуальной оценки является метод **Guided Backpropagation**, описанный в [статье](https://arxiv.org/pdf/1412.6806.pdf). Объяснение метода и код можно найти по [ссылке](https://leslietj.github.io/2020/07/22/Deep-Learning-Guided-BackPropagation/).

<center><img src="https://edunet.kea.su/repo/EduNet-web_dependencies/L10/Guided_Backpropagation.png" alt="alttext" width=600/></center>

<center><p><em>Картинка получена применением модели ResNet18 и Guided Backpropagation</p> </em></center>

Интуитивно нам кажется, что **Guided Backpropagation** дает хорошее объяснение работы модели, потому, что она выделяет границы, по которым легко узнать объект. В статье [Sanity Checks for Saliency Maps](https://arxiv.org/pdf/1810.03292.pdf) показано, что данный метод инвариантен к рандомизации весов верхних слоев модели, что ставит под сомнения его способности объяснения работы модели.

<center><img src="https://edunet.kea.su/repo/EduNet-web_dependencies/L10/guided_backprop_demo.png" alt="alttext" width=900/></center>

<center><p><em>Показан результат Guided Backpropagation  при рандомизации слоев в модели Inception v3, начиная с последнего. 

Source: <a href="https://arxiv.org/pdf/1810.03292.pdf">Sanity Checks for Saliency Maps.</a> </p> </em></center>



В статье [Interpretation of Neural Networks is Fragile](https://arxiv.org/abs/1710.10547) было показано, что небольшое случайное возмущение, добавленное к картинке, не влияющее на результат предсказания, может существенно поменять карты важности:

<center><img src="https://edunet.kea.su/repo/EduNet-web_dependencies/L10/criticism_grad_1.png" alt="alttext" width=500/></center>

<center><p><em>Source: <a href="https://arxiv.org/abs/1710.10547">Interpretation of Neural Networks is Fragile</a></p> </em></center>

В целом [остается проблема](https://arxiv.org/pdf/1912.01451.pdf) оценки качества Saliency Maps. Это не значит что градиентные методы нельзя использовать, это значит, что полезно использовать SHAP или LIME для дополнительной проверки.

# Методы, специфичные для трансформеров

В трансформерах есть механизм **self-attention**, который кажется естественным способом определить, какие токены текста/кусочки изображения имеют большую важность для предсказания. 

<center><img src ="https://edunet.kea.su/repo/EduNet-web_dependencies/L09/self_attention.png"  width="900"></center>

Это бы отлично работало, будь у нас только один блок **self-attention**, но архитектура трансформера состоит из **нескольких блоков кодера/декодера**, составленных друг за другом. От слоя к слою за счет того же **self-attention** **информация в эмбеддингах перемешивается** сильнее и сильнее. Так, например, для классификации текста мы можем использовать эмбеддинг на выходе **BERT**, соответствующий `[CLS]` токену , в котором на входе **BERT** не было никакой информации о последующем тексте.   

 <img src ="https://edunet.kea.su/repo/EduNet-web_dependencies/L08/transformer_text_translation_example.png" width="800">

Source: [The Illustrated BERT, ELMo, and co. (How NLP Cracked Transfer Learning)](https://jalammar.github.io/illustrated-bert/)

Кроме того в стандартном блоке трансформера есть **residual соединения** из-за этого информация о токенах/патчах проходит не только через **attention**, но и через **residual соединения**.

<img src ="https://edunet.kea.su/repo/EduNet-web_dependencies/L08/kolpinsky_sv/transformer_architecture.png" width="400">

По этим причинам объяснение работы трансформера через attention - непростая задача.

**Предупреждение:** Методы объяснения **attention** только частично объясняют работу трансформера, они  разнообразны и могут давать противоречивые результаты. 

Давайте для начала немного поизучаем, как выглядят значения self-attention в трансформерах. Подгружаем библиотеку.

In [ ]:
from IPython.display import clear_output

try:
    import transformers
except ModuleNotFoundError:
    !pip install sentencepiece
    !pip install transformers
    clear_output()
    import transformers

Возьмем базовый русскоязычный разговорный [BERT от Deep Pavlov](https://huggingface.co/DeepPavlov/rubert-base-cased-conversational), [обученный](https://huggingface.co/blanchefort/rubert-base-cased-sentiment) для определения настроения коротких русских текстов. Загружаем токенизатор и модель. Ставим флаг `output_attentions=True`, чтобы модель возвращала значения attention.

In [ ]:
import torch
from transformers import BertTokenizerFast, AutoModelForSequenceClassification

tokenizer = BertTokenizerFast.from_pretrained(
    "blanchefort/rubert-base-cased-sentiment",
)
model = AutoModelForSequenceClassification.from_pretrained(
    "blanchefort/rubert-base-cased-sentiment", 
    output_attentions=True,  # for save attention
) 

Готовим предложения.

In [ ]:
sentences = [
    "Мама мыла раму",
    "Фильм сделан откровенно плохо",
    "Максимально скучный сериал, где сюжет высосан из пальца",
    "Я был в восторге",
    "В общем, кино хорошее и есть много что пообсуждать"
]

tokens = [
    ['[cls]'] + tokenizer.tokenize(sentence)+['[sep]'] for sentence in sentences
]

Посмотрим как разбивается на токены предложение. На выходе токенизатора номера токенов.

In [ ]:
item = 0
print(f"Tokens: {tokens[item]}")
token_ids = [tokenizer.encode(sentence) for sentence in sentences]
print(f"Token ids: {token_ids[item]}")

Посмотрим на предсказания модели, чтобы проверить, насколько она адекватна.

In [ ]:
ans = {
    0: "NEUTRAL",
    1: "POSITIVE",
    2: "NEGATIVE"
}

for item in range(5):
    input_ids = torch.tensor([token_ids[item]])
    model_output = model(input_ids)
    predicted = torch.argmax(model_output.logits, dim=1).numpy()
    print(f"Text: {sentences[item]}")
    print(f"Predict lable = {predicted}, {ans[predicted.item()]}")

В данной моделе 12 слоев (блоков трансформеров), поэтому модель возвращает кортеж из 12 тензоров. Каждый слой имеет 12 голов self-attention. 

In [ ]:
item = 1
input_ids = torch.tensor([token_ids[item]])
model_output = model(input_ids)

attentions = model_output.attentions
print(f"Text: {sentences[item]}")
print(f"Tokens: {tokens[item]}")
print(f"Number of layers: {len(attentions)}")
print(f"Attention size: {attentions[0].shape} "
       "[batch x attention_heads x seq_size x seq_size]")

Преобразуем в однородный массив для удобства манипуляций.

Код этой части лекции основан на [репозитории](https://github.com/samiraabnar/attention_flow). 
[Cтатья](https://arxiv.org/pdf/2005.00928.pdf).


In [ ]:
import numpy as np

def to_array(attentions):
    attentions_arr = [attention.detach().numpy() for attention in attentions]
    return  np.asarray(attentions_arr)[:, 0]

attentions_arr = to_array(attentions)
print(f"Shape: {attentions_arr.shape} "
      "[layers x attention_heads x seq_size x seq_size]")
print(f"Type: {type(attentions_arr)}, Dtype: {attentions_arr.dtype}")

Посмотрим на **нулевую голову нулевого слоя**

In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt

x_ticks = tokens[item]
y_ticks = tokens[item]

sns.heatmap(
    attentions_arr[0][0],
    xticklabels=x_ticks,
    yticklabels=y_ticks,
    cmap="YlOrRd",
)

plt.show()

Тут по **оси x** - токены, **на** которые смотрит внимание, по **оси y** - токены, **куда** записывается результат прохождения слоя. 

Так для слова “плохо” первая голова внимания первого слоя больше всего смотрит на слова “фильм” и “ откровенно”.

Для визуализации внимания можно использовать библиотеку [bertviz](https://pypi.org/project/bertviz/) ([статья](https://arxiv.org/pdf/1904.02679.pdf),
[код](https://colab.research.google.com/drive/1hXIQ77A4TYS4y3UthWF-Ci7V7vVUoxmQ?usp=sharing))

In [ ]:
try:
    import bertviz
except ModuleNotFoundError:
    !pip install bertviz
    clear_output()
    import bertviz

Тут Layer - это выбор слоя, цвета - головы self-attention, слева - куда записывается, справа - на какие токены смотрит. Яркость соединяющих линий - величина attention (чем ярче, тем больше). Картину аналогичную катрине выше можно получить *дважды щелкнув на первый синий квадрат*. 

In [ ]:
from bertviz import head_view

head_view(model_output.attentions, tokens[item])

Усредним значения по головам

In [ ]:
def head_sum(attention_arr):
    return attentions_arr.sum(axis=1)/attentions_arr.shape[1]

attention_head_sum = head_sum(attentions_arr)
print(f"{attention_head_sum.shape} [layers x seq_size x seq_size]")

Посмотрим на усредненное по головам внимание **на первом слое**

In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt

x_ticks = tokens[item]
y_ticks = tokens[item]

sns.heatmap(
    attention_head_sum[0],
    xticklabels=x_ticks,
    yticklabels=y_ticks,
    cmap="YlOrRd",
)

plt.show()

И на **последнем слое**

In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt

x_ticks = tokens[item]
y_ticks = tokens[item]

sns.heatmap(
    attention_head_sum[11],
    xticklabels=x_ticks,
    yticklabels=y_ticks,
    cmap="YlOrRd",
)

plt.show()

По значениям справа от графика видно, что внимание на последнем слое изменяется в **узком диапазоне значений**. 

Посмотрим на **значения внимания для записываемого в [CLS] токен** (эмбеддинг с него используется для классификации).

In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt
import itertools 

x_ticks = tokens[item]
y_ticks = [i for i in range(12, 0, -1)]

sns.heatmap(
    np.flip(attention_head_sum[:, 0, :], axis=0),
    xticklabels=x_ticks,
    yticklabels=y_ticks,
    cmap="YlOrRd"
)

plt.show()

Видно, что после 6-го слоя значения внимания выравниваются. Это связано с тем, что механизм **self-attention** смешивает информацию о токенах.

<font size="5">Residual connection</font>

Давайте сначала определимся, что делать с **residual соединениями**.

<img src ="https://edunet.kea.su/repo/EduNet-web_dependencies/L08/kolpinsky_sv/transformer_architecture.png" width="400">

**Residual соединение** можно записать как

$$ V_{l+1} = V_l + W_{att}V_l$$

где $W_{att}$ - матрица внимания, а $V_l$ - эмбединги.

После нормализации это можно переписать как 

$$A=0.5W{att}+0.5I$$

где $I$ - единичная матрица

[Подробнее](https://arxiv.org/pdf/2005.00928.pdf)

In [ ]:
def residual(attention_head_sum):
     attention_head_sum += np.eye(attention_head_sum.shape[1])[None,...]
     return attention_head_sum / attention_head_sum.sum(axis=-1)[...,None]

attention_res = residual(attention_head_sum)

x_ticks = tokens[item][1:-1]
y_ticks = [i for i in range(12, 0, -1)]

sns.heatmap(
    np.flip(attention_res[:, 0, 1:-1], axis=0),
    xticklabels=x_ticks,
    yticklabels=y_ticks,
    cmap="YlOrRd"
)

plt.show()

## Attention rollout
“Разворачивание внимания” (**Attention rollout**) - предложенный в [статье](https://arxiv.org/pdf/2005.00928.pdf) способ отслеживания информации, распространяемой от входного к выходному блоку, в котором значение внимания рассматривается, как доля пропускаемой информации.  Доли информации перемножаются и суммируются. Итоговая формула - рекурсивное матричное перемножение.

\begin{align}
\widetilde{A}(l_i) = \left\{
\begin{array}{cl}
A(l_i)\widetilde{A}(l_{i-1}) & i>0 \\
A(l_i) & i = 0.
\end{array}
\right.
\end{align}



In [ ]:
def rollout(attention_res):
    rollout_attantion = np.zeros(attention_res.shape)
    rollout_attantion[0] = attention_res[0]
    n_layers = attention_res.shape[0]
    for i in range(1, n_layers):
        rollout_attantion[i] = attention_res[i].dot(rollout_attantion[i-1])
    return rollout_attantion

In [ ]:
rollout_attention = rollout(attention_res)

In [ ]:
x_ticks = tokens[item][1:-1]
y_ticks = [i for i in range(12, 0, -1)]

sns.heatmap(
    np.flip(rollout_attention[:, 0, 1:-1], axis=0),
    xticklabels=x_ticks,
    yticklabels=y_ticks,
    cmap="YlOrRd"
)

plt.show()

Реализацию для ViT и картинок можно найти [тут](https://huggingface.co/spaces/probing-vits/attention-rollout/tree/main) (API упало, но код рабочий, проверено). 

Реализация на pytorch rollout для BERT и некоторых других методов: [API](https://huggingface.co/spaces/amsterdamNLP/attention-rollout), [код](https://huggingface.co/spaces/amsterdamNLP/attention-rollout/tree/main).

## Attention Flow

Другим вариантом рассмотрения распространения внимания является **attention flow**. В нем трансформер представляют в виде направленного графа, **узлы** которого представляют собой **эмбеддинги** между слоями, а **ребра** - связи в виде **attention** с ограниченной емкостью (передающей способностью).

<center><img src ="https://edunet.kea.su/repo/EduNet-web_dependencies/L10/attention_flow.png" width="700">

<p><em>Source: <a href="https://github.com/samiraabnar/attention_flow/blob/master/bert_example.ipynb">Bert Example
</a></p> </em></center>

В такой постановке задача нахождения роли токенов/частей изображения в результирующем эмбеддинге сводится к [задаче о максимальном потоке](https://ru.wikipedia.org/wiki/%D0%97%D0%B0%D0%B4%D0%B0%D1%87%D0%B0_%D0%BE_%D0%BC%D0%B0%D0%BA%D1%81%D0%B8%D0%BC%D0%B0%D0%BB%D1%8C%D0%BD%D0%BE%D0%BC_%D0%BF%D0%BE%D1%82%D0%BE%D0%BA%D0%B5) (задача нахождения такого потока по транспортной сети, что сумма потоков в пункт назначания была максимальна). Это - известная алгоритмическая задача, которую мы не будем разбирать в рамках этого курса.

[Код](https://github.com/samiraabnar/attention_flow), [статья](https://arxiv.org/pdf/2005.00928.pdf).



## Gradient-weighted Attention Rollout

Метод, объединяющий GradCam и Attention Rollout, позволяет оценить положительный и отрицательный вклад токенов / частей изображения в итоговый результат. Предложен в [статье](https://openaccess.thecvf.com/content/CVPR2021/papers/Chefer_Transformer_Interpretability_Beyond_Attention_Visualization_CVPR_2021_paper.pdf), реализацию можно найти тут: [API](https://huggingface.co/spaces/amsterdamNLP/attention-rollout), [код](https://huggingface.co/spaces/amsterdamNLP/attention-rollout/tree/main).



# Заключение

* В ходе урока мы убедились в важности интерпретации работы моделей Искусственного Интеллекта.
* Рассмотрели основные библиотеки, которые используются для интерпретирования: SHAP и LIME. 
* Рассмотрели применение библиотек на примерах:
    - Табличные данные
    - NLP (машинный перевод текста, создание резюме статьи и классификации текстов)
    - CV

Пренебрежение объяснением того, почему модель дала тот или иной результат, ведет к недоверию не только к самой модели, но и к конкретным прогнозам. А, следовательно, является существенным препятствием для дальнейшего введения Вашей идеи в production.



<font size ="6">Список литературы</font>

<font size="5">Книга:</font>

[A Guide for Making Black Box Models Explainable](https://christophm.github.io/interpretable-ml-book/)

<font size="5">Статьи:</font>

[How to Interpret Machine Learning Models with SHAP](https://www.youtube.com/watch?v=m60swo-th4E)

[Бесплатный курс от Kaggle: Machine Learning Explainability](https://www.kaggle.com/learn/machine-learning-explainability)

[EXPLAINABLE AI IN CREDIT RISK MANAGEMENT](https://arxiv.org/pdf/2103.00949v1.pdf)

[Predicting Driver Fatigue in Automated Driving with Explainability](https://arxiv.org/pdf/2103.02162v1.pdf)

[Fooling LIME and SHAP: Adversarial Attacks on Post hoc Explanation Methods](https://arxiv.org/pdf/1911.02508v2.pdf)

[Interpretable Machine Learning](https://christophm.github.io/interpretable-ml-book/intro.html)

<font size="5">SHAP</font>

[Welcome to the SHAP documentation](https://shap.readthedocs.io/en/stable/index.html)

[Git](https://github.com/slundberg/shap)

[A Unified Approach to Interpreting Model Predictions](https://arxiv.org/pdf/1705.07874v2.pdf)

[SHAP (SHapley Additive exPlanations)](https://christophm.github.io/interpretable-ml-book/shap.html)

<font size="5">LIME</font>

[“Why Should I Trust You?” Explaining the Predictions of Any Classifier](https://arxiv.org/pdf/1602.04938.pdf)

[What does LIME really see in images?](https://arxiv.org/pdf/2102.06307v1.pdf)

[Git](https://github.com/marcotcr/lime)

<font size="5">BORUTA</font>

[Feature Selection with the Boruta Package](https://www.researchgate.net/publication/280138095_Feature_Selection_with_Boruta_Package)

[Boruta Explained Exactly How You Wished Someone Explained to You](https://towardsdatascience.com/boruta-explained-the-way-i-wish-someone-explained-it-to-me-4489d70e154a)